# Teknofest Trendyol E-Ticaret için yapay zeka modeli

# Deneme 1 Baseline Oluşturma

**Kod 1** Eğitim ve modelleri kaydetme

In [ ]:
import polars as pl
import pandas as pd
import lightgbm as lgb
import numpy as np
import os

print("--- KOD 1: Model Eğitme ve KAYDETME Süreci Başladı ---")

# --- Adım 1: Veri Yükleme ---
# Colab'a yüklediğiniz dosyanın tam yolunu buraya yazın
TRAIN_PATH = "/content/Trendyol Veri/enriched_train_data_FINAL.parquet"

print(f"Eğitim verisi yükleniyor: {TRAIN_PATH}")
train_full_df = pl.read_parquet(TRAIN_PATH)
print(f"Toplam eğitim verisi boyutu: {train_full_df.shape}")


# --- Adım 2: Veriyi Zamana Göre Ayırma (Train / Validation Split) ---
print("Eğitim verisi, zamana göre sıralanıp yüzdesel olarak ayrılıyor...")
if "ts_hour" in train_full_df.columns and train_full_df["ts_hour"].dtype != pl.Datetime:
    train_full_df = train_full_df.with_columns(pl.col("ts_hour").str.to_datetime())

train_full_df = train_full_df.sort("ts_hour")

# %85 train, %15 valid olarak ayır
split_index = int(len(train_full_df) * 0.85)
train_df = train_full_df[:split_index]
val_df = train_full_df[split_index:]

print(f"Train parçası boyutu: {train_df.shape}")
print(f"Validation parçası boyutu: {val_df.shape}")


# --- Adım 3: Veri Hazırlama ve Tip Dönüşümü ---
print("Veri setleri Pandas'a çevriliyor ve model için hazırlanıyor...")
train_pd = train_df.to_pandas().fillna(0)
val_pd = val_df.to_pandas().fillna(0)

# Object tipli sütunları 'category' tipine çevirerek hatayı gideriyoruz
for col in train_pd.columns:
    if train_pd[col].dtype == 'object':
        train_pd[col] = train_pd[col].astype('category')
        val_pd[col] = val_pd[col].astype('category')

# Modelde kullanılacak özellikleri ve hedefleri belirliyoruz
targets = ["ordered", "clicked"]
# Test setinde olmayan ve hedefle ilişkili sızıntı yapabilecek sütunları hariç tutuyoruz
exclude_cols = targets + ["added_to_cart", "added_to_fav", "ts_hour", "session_id", "user_id_hashed", "content_id_hashed", "search_term_normalized", "level2_category_name", "level1_category_name", "cv_tags", "update_date", "content_creation_date"]
features = [col for col in train_pd.columns if col not in exclude_cols]
cat_features = [col for col in features if isinstance(train_pd[col].dtype, pd.CategoricalDtype)]

print(f"Kullanılacak özellik sayısı: {len(features)}")


# --- Adım 4: lgb.Dataset Oluşturma (RAM Dostu Kısım) ---
print("RAM verimliliği için lgb.Dataset nesneleri oluşturuluyor...")
lgb_train_ordered = lgb.Dataset(train_pd[features], label=train_pd['ordered'], categorical_feature=cat_features)
lgb_valid_ordered = lgb.Dataset(val_pd[features], label=val_pd['ordered'], categorical_feature=cat_features)

lgb_train_clicked = lgb.Dataset(train_pd[features], label=train_pd['clicked'], categorical_feature=cat_features)
lgb_valid_clicked = lgb.Dataset(val_pd[features], label=val_pd['clicked'], categorical_feature=cat_features)


# --- Adım 5: Model Eğitimi ve Kaydetme ---
params = {
    "objective": "binary", "metric": "auc", "boosting_type": "gbdt",
    "learning_rate": 0.03, "num_leaves": 64, "verbose": -1, "seed": 42
}
callbacks = [lgb.early_stopping(100, verbose=True)]

# DÜZELTME: Modelleri kaydetmeden önce klasörün var olduğundan emin ol
MODEL_DIR = "models"
os.makedirs(MODEL_DIR, exist_ok=True)

# 5.1 'ordered' modeli
print("\n'ordered' modeli eğitiliyor...")
ordered_counts = train_pd['ordered'].value_counts()
params['scale_pos_weight'] = ordered_counts[0] / ordered_counts[1]
model_ordered = lgb.train(params, lgb_train_ordered, valid_sets=[lgb_train_ordered, lgb_valid_ordered], valid_names=["train", "valid"], num_boost_round=2000, callbacks=callbacks)
model_ordered.save_model(os.path.join(MODEL_DIR, "model_ordered.txt"))
print("✅ 'ordered' modeli 'model_ordered.txt' olarak kaydedildi.")


# 5.2 'clicked' modeli
print("\n'clicked' modeli eğitiliyor...")
clicked_counts = train_pd['clicked'].value_counts()
params['scale_pos_weight'] = clicked_counts[0] / clicked_counts[1]
model_clicked = lgb.train(params, lgb_train_clicked, valid_sets=[lgb_train_clicked, lgb_valid_clicked], valid_names=["train", "valid"], num_boost_round=2000, callbacks=callbacks)
model_clicked.save_model(os.path.join(MODEL_DIR, "model_clicked.txt"))
print("✅ 'clicked' modeli 'model_clicked.txt' olarak kaydedildi.")

print("\n--- Eğitim ve Kaydetme İşlemi Tamamlandı! ---")

--- KOD 1: Model Eğitme ve KAYDETME Süreci Başladı ---
Eğitim verisi yükleniyor: /content/Trendyol Veri/enriched_train_data_FINAL.parquet
Toplam eğitim verisi boyutu: (2773805, 37)
Eğitim verisi, zamana göre sıralanıp yüzdesel olarak ayrılıyor...
Train parçası boyutu: (2357734, 37)
Validation parçası boyutu: (416071, 37)
Veri setleri Pandas'a çevriliyor ve model için hazırlanıyor...
Kullanılacak özellik sayısı: 23
RAM verimliliği için lgb.Dataset nesneleri oluşturuluyor...

'ordered' modeli eğitiliyor...
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[1]	train's auc: 0.764107	valid's auc: 0.700363
✅ 'ordered' modeli 'model_ordered.txt' olarak kaydedildi.

'clicked' modeli eğitiliyor...
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[142]	train's auc: 0.715502	valid's auc: 0.658479
✅ 'clicked' modeli 'model_clicked.txt' olarak kaydedildi.

--- Eğitim ve Kaydetme İşlemi Tamamlandı! ---


**Kod 2** Tahmini Skor Üretme Ve Kaggle İçin Uygun Submission Dosyası Hazırlama

In [ ]:
import polars as pl
import pandas as pd
import lightgbm as lgb
import numpy as np
# trendyol_metric_group_auc.py dosyasının Colab ortamında olduğundan emin olun
from trendyol_metric_group_auc import score

print("--- KOD 2: Tahmin ve Skorlama Süreci Başladı ---")

# --- Adım 1: Gerekli Veri ve Kaydedilmiş Modelleri Yükleme ---
VAL_PATH = "/content/Trendyol Veri/enriched_train_data_FINAL.parquet"
TEST_PATH = "/content/Trendyol Veri/enriched_test_data_FINAL.parquet"
MODEL_ORDERED_PATH = '/content/models/model_ordered.txt'
MODEL_CLICKED_PATH = '/content/models/model_clicked.txt'

print("Veriler ve kaydedilmiş modeller yükleniyor...")
val_full_df = pl.read_parquet(VAL_PATH)
test_df_pl = pl.read_parquet(TEST_PATH)
model_ordered = lgb.Booster(model_file=MODEL_ORDERED_PATH)
model_clicked = lgb.Booster(model_file=MODEL_CLICKED_PATH)
print("✅ Veriler ve modeller başarıyla yüklendi.")

# --- Adım 2: Validation Seti Hazırlama ve Lokal Skor Hesaplama ---
print("\nLokal skor için validation seti hazırlanıyor...")
if "ts_hour" in val_full_df.columns and val_full_df["ts_hour"].dtype != pl.Datetime:
    val_full_df = val_full_df.with_columns(pl.col("ts_hour").str.to_datetime())
val_full_df = val_full_df.sort("ts_hour")
split_index = int(len(val_full_df) * 0.85)
val_pd = val_full_df[split_index:].to_pandas().fillna(0)

# Özellik listesini eğitimdekiyle tutarlı olacak şekilde belirle
targets = ["ordered", "clicked"]
exclude_cols = targets + ["added_to_cart", "added_to_fav", "ts_hour", "session_id", "user_id_hashed", "content_id_hashed", "search_term_normalized", "level2_category_name", "level1_category_name", "cv_tags", "update_date", "content_creation_date"]
features = [col for col in val_pd.columns if col not in exclude_cols]
for col in features:
    if val_pd[col].dtype == 'object':
        val_pd[col] = val_pd[col].astype('category')

val_pd['p_order'] = model_ordered.predict(val_pd[features], num_iteration=model_ordered.best_iteration)
val_pd['p_click'] = model_clicked.predict(val_pd[features], num_iteration=model_clicked.best_iteration)
val_pd['final_score'] = 0.7 * val_pd['p_order'] + 0.3 * val_pd['p_click']

# Skor fonksiyonu için gerekli DataFrame'leri oluştur
val_solution = val_pd.groupby('session_id').agg(
    ordered_items=('ordered', lambda x: ' '.join(val_pd.loc[x.index][x == 1]['content_id_hashed'])),
    clicked_items=('clicked', lambda x: ' '.join(val_pd.loc[x.index][x == 1]['content_id_hashed'])),
    all_items=('content_id_hashed', ' '.join)
).reset_index()
val_submission = val_pd.sort_values(['session_id', 'final_score'], ascending=[True, False]).groupby('session_id')['content_id_hashed'].apply(' '.join).reset_index()
val_submission.rename(columns={'content_id_hashed': 'prediction'}, inplace=True)

try:
    local_final_score = score(val_solution, val_submission, 'session_id')
    print("\n-------------------------------------------")
    print(f"🏆 LOKAL SKORUNUZ: {local_final_score:.5f}")
    print("-------------------------------------------")
except Exception as e:
    print(f"Skor hesaplanırken bir hata oluştu: {e}")

# --- Adım 3: Kaggle için Submission Dosyası Oluşturma ---
print("\nTest verisi hazırlanıyor ve Kaggle için submission dosyası oluşturuluyor...")
test_pd = test_df_pl.to_pandas().fillna(0)
for col in features:
    if test_pd[col].dtype == 'object':
        test_pd[col] = test_pd[col].astype('category')

p_order_test = model_ordered.predict(test_pd[features], num_iteration=model_ordered.best_iteration)
p_click_test = model_clicked.predict(test_pd[features], num_iteration=model_clicked.best_iteration)

test_df_pl = test_df_pl.with_columns(
    final_score=(0.7 * pl.Series(p_order_test)) + (0.3 * pl.Series(p_click_test))
)

submission_df = test_df_pl.sort(["session_id", "final_score"], descending=True).group_by("session_id").agg(
    pl.col("content_id_hashed").alias("prediction")
).with_columns(
    pl.col("prediction").list.join(" ")
)

# Sağlama kontrolü
expected_rows = 18589
actual_rows = submission_df.shape[0]
print(f"\nOluşturulan submission satır sayısı: {actual_rows} (Beklenen: {expected_rows})")
if actual_rows == expected_rows: print("✅ Satır sayısı doğru.")
else: print("❌ UYARI: Satır sayısı yanlış!")

submission_path = "submission_final_v4.csv"
submission_df.write_csv(submission_path)
print(f"\nSubmission dosyası '{submission_path}' olarak kaydedildi.")

--- KOD 2: Tahmin ve Skorlama Süreci Başladı ---
Veriler ve kaydedilmiş modeller yükleniyor...
✅ Veriler ve modeller başarıyla yüklendi.

Lokal skor için validation seti hazırlanıyor...
Ordered AUC:  0.6506964811575329
Clicked AUC:  0.5915140232878667

-------------------------------------------
🏆 LOKAL SKORUNUZ: 0.63294
-------------------------------------------

Test verisi hazırlanıyor ve Kaggle için submission dosyası oluşturuluyor...

Oluşturulan submission satır sayısı: 18589 (Beklenen: 18589)
✅ Satır sayısı doğru.

Submission dosyası 'submission_final_v4.csv' olarak kaydedildi.


# Deneme 2 Günün Saati / Haftanın Günü Özellikleriyle Eğitim

**Bölüm 1** Yeni eklenen özellik Sütunları ile modelleri eğitmek ve kaydetmek

In [ ]:
import polars as pl
import pandas as pd
import lightgbm as lgb
import numpy as np
import os

print("--- KOD 1: Model Eğitme (Yeni Zamansal Özellikler ile) ---")

# --- Adım 1: Veri Yükleme ---
TRAIN_PATH = "/content/Trendyol Veri/enriched_train_data_FINAL.parquet"
print(f"Eğitim verisi yükleniyor: {TRAIN_PATH}")
train_full_df = pl.read_parquet(TRAIN_PATH)

# --- Adım 2: Veriyi Zamana Göre Ayırma ---
print("Eğitim verisi, zamana göre sıralanıp yüzdesel olarak ayrılıyor...")
if "ts_hour" in train_full_df.columns and train_full_df["ts_hour"].dtype != pl.Datetime:
    train_full_df = train_full_df.with_columns(pl.col("ts_hour").str.to_datetime())

# --- Adım 2.5: YENİ ZAMANSAL ÖZELLİKLER ÜRETME ---
print("Yeni zamansal özellikler üretiliyor: 'hour_of_day' ve 'day_of_week'...")
train_full_df = train_full_df.with_columns(
    pl.col("ts_hour").dt.hour().alias("hour_of_day"),
    pl.col("ts_hour").dt.weekday().alias("day_of_week") # Pazartesi=1, Pazar=7
)

# --- Adım 3: Train / Validation Split ---
train_full_df = train_full_df.sort("ts_hour")
split_index = int(len(train_full_df) * 0.85)
train_df, val_df = train_full_df[:split_index], train_full_df[split_index:]

print(f"Train parçası boyutu: {train_df.shape}")
print(f"Validation parçası boyutu: {val_df.shape}")


# --- Adım 4: Veri Hazırlama ve Tip Dönüşümü ---
print("Veri setleri Pandas'a çevriliyor ve model için hazırlanıyor...")
train_pd = train_df.to_pandas().fillna(0)
val_pd = val_df.to_pandas().fillna(0)

for col in train_pd.columns:
    if train_pd[col].dtype == 'object':
        train_pd[col] = train_pd[col].astype('category')
        val_pd[col] = val_pd[col].astype('category')

targets = ["ordered", "clicked"]
exclude_cols = targets + ["added_to_cart", "added_to_fav", "ts_hour", "session_id", "user_id_hashed", "content_id_hashed", "search_term_normalized", "level2_category_name", "level1_category_name", "cv_tags", "update_date", "content_creation_date"]
features = [col for col in train_pd.columns if col not in exclude_cols]
cat_features = [col for col in features if isinstance(train_pd[col].dtype, pd.CategoricalDtype) or train_pd[col].dtype.name == 'category']

print(f"Kullanılacak özellik sayısı: {len(features)}")


# --- Adım 5: lgb.Dataset Oluşturma ---
print("lgb.Dataset nesneleri oluşturuluyor...")
lgb_train_ordered = lgb.Dataset(train_pd[features], label=train_pd['ordered'], categorical_feature=cat_features)
lgb_valid_ordered = lgb.Dataset(val_pd[features], label=val_pd['ordered'], categorical_feature=cat_features)
lgb_train_clicked = lgb.Dataset(train_pd[features], label=train_pd['clicked'], categorical_feature=cat_features)
lgb_valid_clicked = lgb.Dataset(val_pd[features], label=val_pd['clicked'], categorical_feature=cat_features)


# --- Adım 6: Model Eğitimi ve Kaydetme ---
params = {
    "objective": "binary", "metric": "auc", "boosting_type": "gbdt",
    "learning_rate": 0.03, "num_leaves": 64, "verbose": -1, "seed": 42
}
callbacks = [lgb.early_stopping(100, verbose=True)]

MODEL_DIR = "models_2_doldur_HoursAndDay"
os.makedirs(MODEL_DIR, exist_ok=True)

# 6.1 'ordered' modeli
print("\n'ordered' modeli eğitiliyor...")
ordered_counts = train_pd['ordered'].value_counts()
params['scale_pos_weight'] = ordered_counts[0] / ordered_counts[1]
model_ordered = lgb.train(params, lgb_train_ordered, valid_sets=[lgb_train_ordered, lgb_valid_ordered], valid_names=["train", "valid"], num_boost_round=2000, callbacks=callbacks)
model_ordered.save_model(os.path.join(MODEL_DIR, "model_ordered.txt"))
print(f"✅ 'ordered' modeli '{MODEL_DIR}/model_ordered.txt' olarak kaydedildi.")

# 6.2 'clicked' modeli
print("\n'clicked' modeli eğitiliyor...")
clicked_counts = train_pd['clicked'].value_counts()
params['scale_pos_weight'] = clicked_counts[0] / clicked_counts[1]
model_clicked = lgb.train(params, lgb_train_clicked, valid_sets=[lgb_train_clicked, lgb_valid_clicked], valid_names=["train", "valid"], num_boost_round=2000, callbacks=callbacks)
model_clicked.save_model(os.path.join(MODEL_DIR, "model_clicked.txt"))
print(f"✅ 'clicked' modeli '{MODEL_DIR}/model_clicked_v2.txt' olarak kaydedildi.")

print("\n--- Yeni özelliklerle eğitim ve kaydetme işlemi tamamlandı! ---")

--- KOD 1: Model Eğitme (Yeni Zamansal Özellikler ile) ---
Eğitim verisi yükleniyor: /content/Trendyol Veri/enriched_train_data_FINAL.parquet
Eğitim verisi, zamana göre sıralanıp yüzdesel olarak ayrılıyor...
Yeni zamansal özellikler üretiliyor: 'hour_of_day' ve 'day_of_week'...
Train parçası boyutu: (2357734, 39)
Validation parçası boyutu: (416071, 39)
Veri setleri Pandas'a çevriliyor ve model için hazırlanıyor...
Kullanılacak özellik sayısı: 25
lgb.Dataset nesneleri oluşturuluyor...

'ordered' modeli eğitiliyor...
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[1]	train's auc: 0.764107	valid's auc: 0.700363
✅ 'ordered' modeli 'models_2_doldur_HoursAndDay/model_ordered.txt' olarak kaydedildi.

'clicked' modeli eğitiliyor...
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[122]	train's auc: 0.711014	valid's auc: 0.658447
✅ 'clicked' modeli 'models_2_doldur_HoursAndDay/model_clicked_v2.txt'

**Bölüm 2** Yeni eklenen özellikleri test etme ve uygun submission oluşturma

In [ ]:
import polars as pl
import pandas as pd
import lightgbm as lgb
import numpy as np
import os
# trendyol_metric_group_auc.py dosyasının Colab ortamında olduğundan emin olun
from trendyol_metric_group_auc import score

print("--- KOD 2: Güncellenmiş Tahmin ve Skorlama Süreci ---")

# --- Adım 1: Gerekli Veri ve GÜNCELLENMİŞ Modelleri Yükleme ---
VAL_PATH = "/content/Trendyol Veri/enriched_train_data_FINAL.parquet"
TEST_PATH = "/content/Trendyol Veri/enriched_test_data_FINAL.parquet"
MODEL_DIR = "models_2_doldur_HoursAndDay"
MODEL_ORDERED_PATH = os.path.join(MODEL_DIR, 'model_ordered.txt') # GÜNCELLEME
MODEL_CLICKED_PATH = os.path.join(MODEL_DIR, 'model_clicked.txt') # GÜNCELLEME

print("Veriler ve kaydedilmiş v2 modelleri yükleniyor...")
val_full_df = pl.read_parquet(VAL_PATH)
test_df_pl = pl.read_parquet(TEST_PATH)
model_ordered = lgb.Booster(model_file=MODEL_ORDERED_PATH)
model_clicked = lgb.Booster(model_file=MODEL_CLICKED_PATH)
print("✅ Veriler ve v2 modelleri başarıyla yüklendi.")

# --- Adım 2: Validation Seti Hazırlama (Yeni Özellikler ile) ---
print("\nLokal skor için validation seti hazırlanıyor...")
if "ts_hour" in val_full_df.columns and val_full_df["ts_hour"].dtype != pl.Datetime:
    val_full_df = val_full_df.with_columns(pl.col("ts_hour").str.to_datetime())

# GÜNCELLEME: Eğitimde eklediğimiz zamansal özellikleri buraya da ekliyoruz
val_full_df = val_full_df.with_columns(
    pl.col("ts_hour").dt.hour().alias("hour_of_day"),
    pl.col("ts_hour").dt.weekday().alias("day_of_week")
)

val_full_df = val_full_df.sort("ts_hour")
split_index = int(len(val_full_df) * 0.85)
val_pd = val_full_df[split_index:].to_pandas().fillna(0)

# Özellik listesini eğitimdekiyle tutarlı olacak şekilde belirle
targets = ["ordered", "clicked"]
exclude_cols = targets + ["added_to_cart", "added_to_fav", "ts_hour", "session_id", "user_id_hashed", "content_id_hashed", "search_term_normalized", "level2_category_name", "level1_category_name", "cv_tags", "update_date", "content_creation_date"]
features = [col for col in val_pd.columns if col not in exclude_cols]
for col in features:
    if val_pd[col].dtype == 'object':
        val_pd[col] = val_pd[col].astype('category')

val_pd['p_order'] = model_ordered.predict(val_pd[features], num_iteration=model_ordered.best_iteration)
val_pd['p_click'] = model_clicked.predict(val_pd[features], num_iteration=model_clicked.best_iteration)
val_pd['final_score'] = 0.7 * val_pd['p_order'] + 0.3 * val_pd['p_click']

# Skor fonksiyonu için gerekli DataFrame'leri oluştur
val_solution = val_pd.groupby('session_id').agg(ordered_items=('ordered', lambda x: ' '.join(val_pd.loc[x.index][x == 1]['content_id_hashed'])), clicked_items=('clicked', lambda x: ' '.join(val_pd.loc[x.index][x == 1]['content_id_hashed'])), all_items=('content_id_hashed', ' '.join)).reset_index()
val_submission = val_pd.sort_values(['session_id', 'final_score'], ascending=[True, False]).groupby('session_id')['content_id_hashed'].apply(' '.join).reset_index()
val_submission.rename(columns={'content_id_hashed': 'prediction'}, inplace=True)

try:
    local_final_score = score(val_solution, val_submission, 'session_id')
    print("\n-------------------------------------------")
    print(f"🏆 YENİ MODELLERLE LOKAL SKORUNUZ: {local_final_score:.5f}")
    print("-------------------------------------------")
except Exception as e:
    print(f"Skor hesaplanırken bir hata oluştu: {e}")

# --- Adım 3: Kaggle için Submission Dosyası Oluşturma ---
print("\nTest verisi hazırlanıyor ve Kaggle için submission dosyası oluşturuluyor...")

# GÜNCELLEME: Eğitimde eklediğimiz zamansal özellikleri test setine de ekliyoruz
if "ts_hour" in test_df_pl.columns and test_df_pl["ts_hour"].dtype != pl.Datetime:
    test_df_pl = test_df_pl.with_columns(pl.col("ts_hour").str.to_datetime())
test_df_pl = test_df_pl.with_columns(
    pl.col("ts_hour").dt.hour().alias("hour_of_day"),
    pl.col("ts_hour").dt.weekday().alias("day_of_week")
)

test_pd = test_df_pl.to_pandas().fillna(0)
for col in features:
    if test_pd[col].dtype == 'object':
        test_pd[col] = test_pd[col].astype('category')

p_order_test = model_ordered.predict(test_pd[features], num_iteration=model_ordered.best_iteration)
p_click_test = model_clicked.predict(test_pd[features], num_iteration=model_clicked.best_iteration)

test_df_pl = test_df_pl.with_columns(
    final_score=(0.7 * pl.Series(p_order_test)) + (0.3 * pl.Series(p_click_test))
)

submission_df = test_df_pl.sort(["session_id", "final_score"], descending=True).group_by("session_id").agg(
    pl.col("content_id_hashed").alias("prediction")
).with_columns(
    pl.col("prediction").list.join(" ")
)

# Sağlama kontrolü
expected_rows = 18589
actual_rows = submission_df.shape[0]
print(f"\nOluşturulan submission satır sayısı: {actual_rows} (Beklenen: {expected_rows})")
if actual_rows == expected_rows: print("✅ Satır sayısı doğru.")
else: print("❌ UYARI: Satır sayısı yanlış!")

submission_path = "submission_final_v5.csv" # Versiyonu güncelledim
submission_df.write_csv(submission_path)
print(f"\nSubmission dosyası '{submission_path}' olarak kaydedildi.")

--- KOD 2: Güncellenmiş Tahmin ve Skorlama Süreci ---
Veriler ve kaydedilmiş v2 modelleri yükleniyor...
✅ Veriler ve v2 modelleri başarıyla yüklendi.

Lokal skor için validation seti hazırlanıyor...
Ordered AUC:  0.6504508564295987
Clicked AUC:  0.5918174552587637

-------------------------------------------
🏆 YENİ MODELLERLE LOKAL SKORUNUZ: 0.63286
-------------------------------------------

Test verisi hazırlanıyor ve Kaggle için submission dosyası oluşturuluyor...

Oluşturulan submission satır sayısı: 18589 (Beklenen: 18589)
✅ Satır sayısı doğru.

Submission dosyası 'submission_final_v5.csv' olarak kaydedildi.


---------------------------------
Başka Bir Deneme
---------------------------------

In [ ]:
import polars as pl
import pandas as pd
import lightgbm as lgb
import numpy as np
import os

print("--- KOD 1: Nihai Model Eğitme ve Kaydetme Süreci ---")

# --- Adım 1: Veri Yükleme ---
TRAIN_PATH = "/content/Trendyol Veri/enriched_train_data_FINAL.parquet"
print(f"Eğitim verisi yükleniyor: {TRAIN_PATH}")
train_full_df = pl.read_parquet(TRAIN_PATH)
print(f"Toplam eğitim verisi boyutu: {train_full_df.shape}")


# --- Adım 2: Özellik Mühendisliği ve Veri Ayırma ---
print("Yeni zamansal özellikler üretiliyor ve veri ayrılıyor...")
if "ts_hour" in train_full_df.columns and train_full_df["ts_hour"].dtype != pl.Datetime:
    train_full_df = train_full_df.with_columns(pl.col("ts_hour").str.to_datetime())

# 2.1 Zamansal Özellikleri Üretme
train_full_df = train_full_df.with_columns(
    pl.col("ts_hour").dt.hour().alias("hour_of_day"),
    pl.col("ts_hour").dt.weekday().alias("day_of_week")
)

# 2.2 Veriyi Zamana Göre Train/Validation Olarak Ayırma
train_full_df = train_full_df.sort("ts_hour")
split_index = int(len(train_full_df) * 0.85)
train_df, val_df = train_full_df[:split_index], train_full_df[split_index:]

print(f"Train parçası boyutu: {train_df.shape}")
print(f"Validation parçası boyutu: {val_df.shape}")


# --- Adım 3: Veri Hazırlama ve Tip Dönüşümü ---
print("Veri setleri Pandas'a çevriliyor ve model için hazırlanıyor...")
train_pd = train_df.to_pandas().fillna(0)
val_pd = val_df.to_pandas().fillna(0)

# Kategorik olarak ele alınacak sütunları belirliyoruz
string_cols = [col for col in train_pd.columns if train_pd[col].dtype == 'object']
temporal_cols_as_cat = ["hour_of_day", "day_of_week"]
cols_to_categorize = string_cols + temporal_cols_as_cat

# Belirlenen sütunların tipini 'category' olarak değiştiriyoruz
for col in cols_to_categorize:
    if col in train_pd.columns:
        train_pd[col] = train_pd[col].astype('category')
        val_pd[col] = val_pd[col].astype('category')

# Modelde kullanılacak özellikleri ve hedefleri belirliyoruz
targets = ["ordered", "clicked"]
exclude_cols = targets + ["added_to_cart", "added_to_fav", "ts_hour", "session_id", "user_id_hashed", "content_id_hashed", "search_term_normalized", "level2_category_name", "level1_category_name", "cv_tags", "update_date", "content_creation_date"]
features = [col for col in train_pd.columns if col not in exclude_cols]
cat_features = [col for col in features if isinstance(train_pd[col].dtype, pd.CategoricalDtype)]

print(f"Kullanılacak özellik sayısı: {len(features)}")
print(f"Kategorik olarak belirlenen özellik sayısı: {len(cat_features)}")


# --- Adım 4: lgb.Dataset Oluşturma ---
print("lgb.Dataset nesneleri oluşturuluyor...")
lgb_train_ordered = lgb.Dataset(train_pd[features], label=train_pd['ordered'], categorical_feature=cat_features)
lgb_valid_ordered = lgb.Dataset(val_pd[features], label=val_pd['ordered'], categorical_feature=cat_features)

lgb_train_clicked = lgb.Dataset(train_pd[features], label=train_pd['clicked'], categorical_feature=cat_features)
lgb_valid_clicked = lgb.Dataset(val_pd[features], label=val_pd['clicked'], categorical_feature=cat_features)


# --- Adım 5: Model Eğitimi ve Kaydetme ---
params = {
    "objective": "binary", "metric": "auc", "boosting_type": "gbdt",
    "learning_rate": 0.03, "num_leaves": 64, "verbose": -1, "seed": 42
}
callbacks = [lgb.early_stopping(100, verbose=True)]

MODEL_DIR = "models_3_doldur_HoursAndDayCategory"
os.makedirs(MODEL_DIR, exist_ok=True)

# 5.1 'ordered' modeli
print("\n'ordered' modeli eğitiliyor...")
ordered_counts = train_pd['ordered'].value_counts()
params['scale_pos_weight'] = ordered_counts[0] / ordered_counts[1]
model_ordered = lgb.train(params, lgb_train_ordered, valid_sets=[lgb_train_ordered, lgb_valid_ordered], valid_names=["train", "valid"], num_boost_round=2000, callbacks=callbacks)
model_ordered.save_model(os.path.join(MODEL_DIR, "model_ordered.txt"))
print(f"✅ 'ordered' modeli '{MODEL_DIR}/model_ordered.txt' olarak kaydedildi.")

# 5.2 'clicked' modeli
print("\n'clicked' modeli eğitiliyor...")
clicked_counts = train_pd['clicked'].value_counts()
params['scale_pos_weight'] = clicked_counts[0] / clicked_counts[1]
model_clicked = lgb.train(params, lgb_train_clicked, valid_sets=[lgb_train_clicked, lgb_valid_clicked], valid_names=["train", "valid"], num_boost_round=2000, callbacks=callbacks)
model_clicked.save_model(os.path.join(MODEL_DIR, "model_clicked.txt"))
print(f"✅ 'clicked' modeli '{MODEL_DIR}/model_clicked.txt' olarak kaydedildi.")

print("\n--- Eğitim ve Kaydetme İşlemi Tamamlandı! ---")

--- KOD 1: Nihai Model Eğitme ve Kaydetme Süreci ---
Eğitim verisi yükleniyor: /content/Trendyol Veri/enriched_train_data_FINAL.parquet
Toplam eğitim verisi boyutu: (2773805, 37)
Yeni zamansal özellikler üretiliyor ve veri ayrılıyor...
Train parçası boyutu: (2357734, 39)
Validation parçası boyutu: (416071, 39)
Veri setleri Pandas'a çevriliyor ve model için hazırlanıyor...
Kullanılacak özellik sayısı: 25
Kategorik olarak belirlenen özellik sayısı: 4
lgb.Dataset nesneleri oluşturuluyor...

'ordered' modeli eğitiliyor...
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[1]	train's auc: 0.764107	valid's auc: 0.700363
✅ 'ordered' modeli 'models_3_doldur_HoursAndDayCategory/model_ordered.txt' olarak kaydedildi.

'clicked' modeli eğitiliyor...
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[127]	train's auc: 0.712264	valid's auc: 0.659234
✅ 'clicked' modeli 'models_3_doldur_HoursAndDayCategory/mo

In [ ]:
import polars as pl
import pandas as pd
import lightgbm as lgb
import numpy as np
import os
# trendyol_metric_group_auc.py dosyasının Colab ortamında olduğundan emin olun
from trendyol_metric_group_auc import score

print("--- KOD 2: Nihai Tahmin ve Skorlama Süreci ---")

# --- Adım 1: Gerekli Veri ve Kaydedilmiş Modelleri Yükleme ---
VAL_PATH = "/content/Trendyol Veri/enriched_train_data_FINAL.parquet"
TEST_PATH = "/content/Trendyol Veri/enriched_test_data_FINAL.parquet"
MODEL_DIR = "models_3_doldur_HoursAndDayCategory"
MODEL_ORDERED_PATH = os.path.join(MODEL_DIR, 'model_ordered.txt')
MODEL_CLICKED_PATH = os.path.join(MODEL_DIR, 'model_clicked.txt')

print("Veriler ve kaydedilmiş modeller yükleniyor...")
val_full_df = pl.read_parquet(VAL_PATH)
test_df_pl = pl.read_parquet(TEST_PATH)
model_ordered = lgb.Booster(model_file=MODEL_ORDERED_PATH)
model_clicked = lgb.Booster(model_file=MODEL_CLICKED_PATH)
print("✅ Veriler ve modeller başarıyla yüklendi.")


# --- Adım 2: Lokal Skor İçin Validation Seti Hazırlama ---
print("\nLokal skor için validation seti hazırlanıyor...")
if "ts_hour" in val_full_df.columns and val_full_df["ts_hour"].dtype != pl.Datetime:
    val_full_df = val_full_df.with_columns(pl.col("ts_hour").str.to_datetime())

# Yeni zamansal özellikleri ekle
val_full_df = val_full_df.with_columns(
    pl.col("ts_hour").dt.hour().alias("hour_of_day"),
    pl.col("ts_hour").dt.weekday().alias("day_of_week")
)

val_full_df = val_full_df.sort("ts_hour")
split_index = int(len(val_full_df) * 0.85)
val_pd = val_full_df[split_index:].to_pandas().fillna(0)

# Özellik listesini ve kategorik tipleri eğitimdekiyle tutarlı yap
targets = ["ordered", "clicked"]
exclude_cols = targets + ["added_to_cart", "added_to_fav", "ts_hour", "session_id", "user_id_hashed", "content_id_hashed", "search_term_normalized", "level2_category_name", "level1_category_name", "cv_tags", "update_date", "content_creation_date"]
features = [col for col in val_pd.columns if col not in exclude_cols]

string_cols = [col for col in val_pd.columns if val_pd[col].dtype == 'object']
temporal_cols_as_cat = ["hour_of_day", "day_of_week"]
cols_to_categorize = string_cols + temporal_cols_as_cat
for col in cols_to_categorize:
    if col in val_pd.columns:
        val_pd[col] = val_pd[col].astype('category')

# Lokal skor için tahmin üret
val_pd['p_order'] = model_ordered.predict(val_pd[features], num_iteration=model_ordered.best_iteration)
val_pd['p_click'] = model_clicked.predict(val_pd[features], num_iteration=model_clicked.best_iteration)
val_pd['final_score'] = 0.7 * val_pd['p_order'] + 0.3 * val_pd['p_click']

# Skor fonksiyonu için gerekli DataFrame'leri oluştur
val_solution = val_pd.groupby('session_id').agg(ordered_items=('ordered', lambda x: ' '.join(val_pd.loc[x.index][x == 1]['content_id_hashed'])), clicked_items=('clicked', lambda x: ' '.join(val_pd.loc[x.index][x == 1]['content_id_hashed'])), all_items=('content_id_hashed', ' '.join)).reset_index()
val_submission = val_pd.sort_values(['session_id', 'final_score'], ascending=[True, False]).groupby('session_id')['content_id_hashed'].apply(' '.join).reset_index()
val_submission.rename(columns={'content_id_hashed': 'prediction'}, inplace=True)

try:
    local_final_score = score(val_solution, val_submission, 'session_id')
    print("\n-------------------------------------------")
    print(f"🏆 LOKAL SKORUNUZ: {local_final_score:.5f}")
    print("-------------------------------------------")
except Exception as e:
    print(f"Skor hesaplanırken bir hata oluştu: {e}")


# --- Adım 3: Kaggle için Submission Dosyası Oluşturma ---
print("\nTest verisi hazırlanıyor ve Kaggle için submission dosyası oluşturuluyor...")
if "ts_hour" in test_df_pl.columns and test_df_pl["ts_hour"].dtype != pl.Datetime:
    test_df_pl = test_df_pl.with_columns(pl.col("ts_hour").str.to_datetime())
test_df_pl = test_df_pl.with_columns(
    pl.col("ts_hour").dt.hour().alias("hour_of_day"),
    pl.col("ts_hour").dt.weekday().alias("day_of_week")
)
test_pd = test_df_pl.to_pandas().fillna(0)
for col in cols_to_categorize:
    if col in test_pd.columns:
        test_pd[col] = test_pd[col].astype('category')

p_order_test = model_ordered.predict(test_pd[features], num_iteration=model_ordered.best_iteration)
p_click_test = model_clicked.predict(test_pd[features], num_iteration=model_clicked.best_iteration)

test_df_pl = test_df_pl.with_columns(
    final_score=(0.7 * pl.Series(p_order_test)) + (0.3 * pl.Series(p_click_test))
)
submission_df = test_df_pl.sort(["session_id", "final_score"], descending=True).group_by("session_id").agg(
    pl.col("content_id_hashed").alias("prediction")
).with_columns(
    pl.col("prediction").list.join(" ")
)

# Sağlama kontrolü
expected_rows = 18589
actual_rows = submission_df.shape[0]
print(f"\nOluşturulan submission satır sayısı: {actual_rows} (Beklenen: {expected_rows})")
if actual_rows == expected_rows: print("✅ Satır sayısı doğru.")
else: print("❌ UYARI: Satır sayısı yanlış!")

submission_path = "submission.csv"
submission_df.write_csv(submission_path)
print(f"\nSubmission dosyası '{submission_path}' olarak kaydedildi.")

--- KOD 2: Nihai Tahmin ve Skorlama Süreci ---
Veriler ve kaydedilmiş modeller yükleniyor...
✅ Veriler ve modeller başarıyla yüklendi.

Lokal skor için validation seti hazırlanıyor...


/tmp/ipython-input-3194590946.py:59: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  val_solution = val_pd.groupby('session_id').agg(ordered_items=('ordered', lambda x: ' '.join(val_pd.loc[x.index][x == 1]['content_id_hashed'])), clicked_items=('clicked', lambda x: ' '.join(val_pd.loc[x.index][x == 1]['content_id_hashed'])), all_items=('content_id_hashed', ' '.join)).reset_index()
/tmp/ipython-input-3194590946.py:60: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  val_submission = val_pd.sort_values(['session_id', 'final_score'], ascending=[True, False]).groupby('session_id')['content_id_hashed'].apply(' '.join).

Ordered AUC:  0.6504243273845817
Clicked AUC:  0.5914654200400459

-------------------------------------------
🏆 LOKAL SKORUNUZ: 0.63274
-------------------------------------------

Test verisi hazırlanıyor ve Kaggle için submission dosyası oluşturuluyor...

Oluşturulan submission satır sayısı: 18589 (Beklenen: 18589)
✅ Satır sayısı doğru.

Submission dosyası 'submission.csv' olarak kaydedildi.


# Deneme 3 Fiyat Uygunluğu

**Kod 1** Fiyat uygunluğuna göre eğitim

In [ ]:
import polars as pl
import pandas as pd
import lightgbm as lgb
import numpy as np
import os

print("--- KOD 1: Model Eğitme (Ayrı Özellik Setleriyle) ---")

# --- Adım 1: Veri Yükleme ---
TRAIN_PATH = "/content/Trendyol Veri/enriched_train_data_FINAL.parquet"
train_full_df = pl.read_parquet(TRAIN_PATH)
print(f"Toplam eğitim verisi boyutu: {train_full_df.shape}")

# --- Adım 2: Ön Özellikler ve Train/Validation Ayrımı ---
print("Ön özellikler üretiliyor ve veri zamana göre ayrılıyor...")
if "ts_hour" in train_full_df.columns and train_full_df["ts_hour"].dtype != pl.Datetime:
    train_full_df = train_full_df.with_columns(pl.col("ts_hour").str.to_datetime())
train_full_df = train_full_df.with_columns(
    pl.col("ts_hour").dt.hour().alias("hour_of_day"),
    pl.col("ts_hour").dt.weekday().alias("day_of_week")
)
train_full_df = train_full_df.sort("ts_hour")
split_index = int(len(train_full_df) * 0.85)
train_df, val_df = train_full_df[:split_index], train_full_df[split_index:]

# --- Adım 3: Etkileşim Özelliğini SADECE Train Setinden Hesaplama ---
print("Fiyat etkileşim özelliği, SADECE train verisi üzerinden hesaplanıyor...")
price_features_lazy = pl.scan_parquet(TRAIN_PATH).select(["content_id_hashed", "selling_price", "update_date"])
latest_price_features = price_features_lazy.sort("update_date", descending=True).group_by("content_id_hashed").first().collect()
user_orders = train_df.filter(pl.col("ordered") == 1).select(["user_id_hashed", "content_id_hashed"])
user_orders_with_price = user_orders.join(latest_price_features, on="content_id_hashed", how="left")
agg_user_price_features = user_orders_with_price.group_by("user_id_hashed").agg(pl.mean("selling_price").alias("user_avg_order_price"))
FEATURES_DIR = "features"
os.makedirs(FEATURES_DIR, exist_ok=True)
agg_user_price_features.write_parquet(os.path.join(FEATURES_DIR, "user_price_features_final.parquet"))
print(f"✅ Sızıntısız kullanıcı fiyat özellikleri kaydedildi.")

# --- Adım 4: Özellikleri Ekleme ve Akıllı Boşluk Doldurma ---
global_avg_order_price = agg_user_price_features.select(pl.mean("user_avg_order_price")).item()
train_df = train_df.join(agg_user_price_features, on="user_id_hashed", how="left").with_columns(pl.col("user_avg_order_price").fill_null(global_avg_order_price)).with_columns((pl.col("selling_price") / pl.col("user_avg_order_price")).alias("price_vs_user_avg_ratio"))
val_df = val_df.join(agg_user_price_features, on="user_id_hashed", how="left").with_columns(pl.col("user_avg_order_price").fill_null(global_avg_order_price)).with_columns((pl.col("selling_price") / pl.col("user_avg_order_price")).alias("price_vs_user_avg_ratio"))

# --- Adım 5: Veri Hazırlama ve AYRI ÖZELLİK SETLERİ OLUŞTURMA ---
print("Veri setleri Pandas'a çevriliyor ve ayrı özellik setleri oluşturuluyor...")
train_pd = train_df.to_pandas().fillna(0)
val_pd = val_df.to_pandas().fillna(0)
string_cols = [col for col in train_pd.columns if train_pd[col].dtype == 'object']
temporal_cols_as_cat = ["hour_of_day", "day_of_week"]
cols_to_categorize = string_cols + temporal_cols_as_cat
for col in cols_to_categorize:
    if col in train_pd.columns:
        train_pd[col], val_pd[col] = train_pd[col].astype('category'), val_pd[col].astype('category')

# 5.1 Temel Özellik Listesi
targets = ["ordered", "clicked"]
exclude_cols = targets + ["added_to_cart", "added_to_fav", "ts_hour", "session_id", "user_id_hashed", "content_id_hashed", "search_term_normalized", "level2_category_name", "level1_category_name", "cv_tags", "update_date", "content_creation_date"]
features_base = [col for col in train_pd.columns if col not in exclude_cols]

# 5.2 Modele Özel Özellik Listeleri
price_features_to_exclude = ["user_avg_order_price", "price_vs_user_avg_ratio"]
features_for_ordered = [f for f in features_base if f not in price_features_to_exclude]
features_for_clicked = features_base # clicked modeli tüm özellikleri kullanabilir

cat_features_ordered = [f for f in features_for_ordered if isinstance(train_pd[f].dtype, pd.CategoricalDtype)]
cat_features_clicked = [f for f in features_for_clicked if isinstance(train_pd[f].dtype, pd.CategoricalDtype)]

print(f"'ordered' modeli için kullanılacak özellik sayısı: {len(features_for_ordered)}")
print(f"'clicked' modeli için kullanılacak özellik sayısı: {len(features_for_clicked)}")

# --- Adım 6: lgb.Dataset Oluşturma (Ayrı Ayrı) ---
print("Modele özel lgb.Dataset nesneleri oluşturuluyor...")
lgb_train_ordered = lgb.Dataset(train_pd[features_for_ordered], label=train_pd['ordered'], categorical_feature=cat_features_ordered)
lgb_valid_ordered = lgb.Dataset(val_pd[features_for_ordered], label=val_pd['ordered'], categorical_feature=cat_features_ordered)

lgb_train_clicked = lgb.Dataset(train_pd[features_for_clicked], label=train_pd['clicked'], categorical_feature=cat_features_clicked)
lgb_valid_clicked = lgb.Dataset(val_pd[features_for_clicked], label=val_pd['clicked'], categorical_feature=cat_features_clicked)

# --- Adım 7: Model Eğitimi ve Kaydetme ---
params = { "objective": "binary", "metric": "auc", "boosting_type": "gbdt", "learning_rate": 0.03, "num_leaves": 64, "verbose": -1, "seed": 42 }
callbacks = [lgb.early_stopping(100, verbose=True)]
MODEL_DIR = "models_5_doldur_FiyatUygunlukAyrı"
os.makedirs(MODEL_DIR, exist_ok=True)

print("\n'ordered' modeli (sınırlı özelliklerle) eğitiliyor...")
ordered_counts = train_pd['ordered'].value_counts()
params['scale_pos_weight'] = ordered_counts[0] / ordered_counts[1]
model_ordered = lgb.train(params, lgb_train_ordered, valid_sets=[lgb_train_ordered, lgb_valid_ordered], valid_names=["train", "valid"], num_boost_round=2000, callbacks=callbacks)
model_ordered.save_model(os.path.join(MODEL_DIR, "model_ordered_final_split.txt"))
print(f"✅ 'ordered' modeli '{MODEL_DIR}/model_ordered_final_split.txt' olarak kaydedildi.")

print("\n'clicked' modeli (tüm özelliklerle) eğitiliyor...")
clicked_counts = train_pd['clicked'].value_counts()
params['scale_pos_weight'] = clicked_counts[0] / clicked_counts[1]
model_clicked = lgb.train(params, lgb_train_clicked, valid_sets=[lgb_train_clicked, lgb_valid_clicked], valid_names=["train", "valid"], num_boost_round=2000, callbacks=callbacks)
model_clicked.save_model(os.path.join(MODEL_DIR, "model_clicked_final_split.txt"))
print(f"✅ 'clicked' modeli '{MODEL_DIR}/model_clicked_final_split.txt' olarak kaydedildi.")

print("\n--- Eğitim ve Kaydetme İşlemi Tamamlandı! ---")

--- KOD 1: Model Eğitme (Ayrı Özellik Setleriyle) ---
Toplam eğitim verisi boyutu: (2773805, 37)
Ön özellikler üretiliyor ve veri zamana göre ayrılıyor...
Fiyat etkileşim özelliği, SADECE train verisi üzerinden hesaplanıyor...
✅ Sızıntısız kullanıcı fiyat özellikleri kaydedildi.
Veri setleri Pandas'a çevriliyor ve ayrı özellik setleri oluşturuluyor...
'ordered' modeli için kullanılacak özellik sayısı: 25
'clicked' modeli için kullanılacak özellik sayısı: 27
Modele özel lgb.Dataset nesneleri oluşturuluyor...

'ordered' modeli (sınırlı özelliklerle) eğitiliyor...
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[1]	train's auc: 0.764107	valid's auc: 0.700363
✅ 'ordered' modeli 'models_5_doldur_FiyatUygunlukAyrı/model_ordered_final_split.txt' olarak kaydedildi.

'clicked' modeli (tüm özelliklerle) eğitiliyor...
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[122]	train's auc: 0.732438	valid's

**Kod 2** Fiyat uygunluğuna göre test ve submission oluşturma

In [ ]:
import polars as pl
import pandas as pd
import lightgbm as lgb
import numpy as np
import os
from trendyol_metric_group_auc import score

print("--- KOD 2: Nihai Tahmin ve Skorlama (Ayrı Özellik Setleriyle) ---")

# --- Adım 1: Gerekli Dosyaları Yükleme ---
VAL_PATH = "/content/Trendyol Veri/enriched_train_data_FINAL.parquet"
TEST_PATH = "/content/Trendyol Veri/enriched_test_data_FINAL.parquet"
FEATURES_DIR = "features"
MODELS_DIR = "models_5_doldur_FiyatUygunlukAyrı"
USER_PRICE_FEATURES_PATH = os.path.join(FEATURES_DIR, "user_price_features_final.parquet")
MODEL_ORDERED_PATH = os.path.join(MODELS_DIR, 'model_ordered_final_split.txt')
MODEL_CLICKED_PATH = os.path.join(MODELS_DIR, 'model_clicked_final_split.txt')

print("Veriler, özellikler ve kaydedilmiş nihai modeller yükleniyor...")
val_full_df = pl.read_parquet(VAL_PATH)
test_df_pl = pl.read_parquet(TEST_PATH)
user_price_features = pl.read_parquet(USER_PRICE_FEATURES_PATH)
model_ordered = lgb.Booster(model_file=MODEL_ORDERED_PATH)
model_clicked = lgb.Booster(model_file=MODEL_CLICKED_PATH)
print("✅ Yükleme tamamlandı.")

# --- Adım 2: Lokal Skor İçin Validation Seti Hazırlama ---
print("\nLokal skor için validation seti hazırlanıyor...")
if "ts_hour" in val_full_df.columns and val_full_df["ts_hour"].dtype != pl.Datetime:
    val_full_df = val_full_df.with_columns(pl.col("ts_hour").str.to_datetime())
val_full_df = val_full_df.with_columns(
    pl.col("ts_hour").dt.hour().alias("hour_of_day"),
    pl.col("ts_hour").dt.weekday().alias("day_of_week")
)
global_avg_order_price = user_price_features.select(pl.mean("user_avg_order_price")).item()
val_full_df = val_full_df.join(user_price_features, on="user_id_hashed", how="left").with_columns(pl.col("user_avg_order_price").fill_null(global_avg_order_price)).with_columns((pl.col("selling_price") / pl.col("user_avg_order_price")).alias("price_vs_user_avg_ratio"))

val_full_df = val_full_df.sort("ts_hour")
split_index = int(len(val_full_df) * 0.85)
val_pd = val_full_df[split_index:].to_pandas().fillna(0)

# Ayrı özellik setlerini burada da tanımla
targets = ["ordered", "clicked"]
exclude_cols = targets + ["added_to_cart", "added_to_fav", "ts_hour", "session_id", "user_id_hashed", "content_id_hashed", "search_term_normalized", "level2_category_name", "level1_category_name", "cv_tags", "update_date", "content_creation_date"]
features_base = [col for col in val_pd.columns if col not in exclude_cols]
price_features_to_exclude = ["user_avg_order_price", "price_vs_user_avg_ratio"]
features_for_ordered = [f for f in features_base if f not in price_features_to_exclude]
features_for_clicked = features_base
string_cols = [col for col in val_pd.columns if val_pd[col].dtype == 'object']
temporal_cols_as_cat = ["hour_of_day", "day_of_week"]
cols_to_categorize = string_cols + temporal_cols_as_cat
for col in cols_to_categorize:
    if col in val_pd.columns:
        val_pd[col] = val_pd[col].astype('category')

# Tahmin yaparken doğru özellik setini kullan
val_pd['p_order'] = model_ordered.predict(val_pd[features_for_ordered], num_iteration=model_ordered.best_iteration)
val_pd['p_click'] = model_clicked.predict(val_pd[features_for_clicked], num_iteration=model_clicked.best_iteration)
val_pd['final_score'] = 0.7 * val_pd['p_order'] + 0.3 * val_pd['p_click']

# Skorlama ve submission (bu kısımlar aynı)
val_solution = val_pd.groupby('session_id').agg(ordered_items=('ordered', lambda x: ' '.join(val_pd.loc[x.index][x == 1]['content_id_hashed'])), clicked_items=('clicked', lambda x: ' '.join(val_pd.loc[x.index][x == 1]['content_id_hashed'])), all_items=('content_id_hashed', ' '.join)).reset_index()
val_submission = val_pd.sort_values(['session_id', 'final_score'], ascending=[True, False]).groupby('session_id')['content_id_hashed'].apply(' '.join).reset_index()
val_submission.rename(columns={'content_id_hashed': 'prediction'}, inplace=True)
try:
    local_final_score = score(val_solution, val_submission, 'session_id')
    print("\n-------------------------------------------")
    print(f"🏆 LOKAL SKORUNUZ (Ayrı Özelliklerle): {local_final_score:.5f}")
    print("-------------------------------------------")
except Exception as e:
    print(f"Skor hesaplanırken bir hata oluştu: {e}")

# --- Adım 3: Kaggle için Submission Dosyası Oluşturma ---
print("\nTest verisi hazırlanıyor ve Kaggle için submission dosyası oluşturuluyor...")
if "ts_hour" in test_df_pl.columns and test_df_pl["ts_hour"].dtype != pl.Datetime:
    test_df_pl = test_df_pl.with_columns(pl.col("ts_hour").str.to_datetime())
test_df_pl = test_df_pl.with_columns(pl.col("ts_hour").dt.hour().alias("hour_of_day"), pl.col("ts_hour").dt.weekday().alias("day_of_week"))
test_df_pl = test_df_pl.join(user_price_features, on="user_id_hashed", how="left").with_columns(pl.col("user_avg_order_price").fill_null(global_avg_order_price)).with_columns((pl.col("selling_price") / pl.col("user_avg_order_price")).alias("price_vs_user_avg_ratio"))
test_pd = test_df_pl.to_pandas().fillna(0)
for col in cols_to_categorize:
    if col in test_pd.columns:
        test_pd[col] = test_pd[col].astype('category')

# Tahmin yaparken doğru özellik setini kullan
p_order_test = model_ordered.predict(test_pd[features_for_ordered], num_iteration=model_ordered.best_iteration)
p_click_test = model_clicked.predict(test_pd[features_for_clicked], num_iteration=model_clicked.best_iteration)

test_df_pl = test_df_pl.with_columns(final_score=(0.7 * pl.Series(p_order_test)) + (0.3 * pl.Series(p_click_test)))
submission_df = test_df_pl.sort(["session_id", "final_score"], descending=True).group_by("session_id").agg(pl.col("content_id_hashed").alias("prediction")).with_columns(pl.col("prediction").list.join(" "))
expected_rows = 18589
actual_rows = submission_df.shape[0]
print(f"\nOluşturulan submission satır sayısı: {actual_rows} (Beklenen: {expected_rows})")
if actual_rows == expected_rows: print("✅ Satır sayısı doğru.")
else: print("❌ UYARI: Satır sayısı yanlış!")
submission_path = "submission.csv"
submission_df.write_csv(submission_path)
print(f"\nSubmission dosyası '{submission_path}' olarak kaydedildi.")

--- KOD 2: Nihai Tahmin ve Skorlama (Ayrı Özellik Setleriyle) ---
Veriler, özellikler ve kaydedilmiş nihai modeller yükleniyor...
✅ Yükleme tamamlandı.

Lokal skor için validation seti hazırlanıyor...


/tmp/ipython-input-1547542627.py:62: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  val_solution = val_pd.groupby('session_id').agg(ordered_items=('ordered', lambda x: ' '.join(val_pd.loc[x.index][x == 1]['content_id_hashed'])), clicked_items=('clicked', lambda x: ' '.join(val_pd.loc[x.index][x == 1]['content_id_hashed'])), all_items=('content_id_hashed', ' '.join)).reset_index()
/tmp/ipython-input-1547542627.py:63: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  val_submission = val_pd.sort_values(['session_id', 'final_score'], ascending=[True, False]).groupby('session_id')['content_id_hashed'].apply(' '.join).

Ordered AUC:  0.6501352808701953
Clicked AUC:  0.5901990785840607

-------------------------------------------
🏆 LOKAL SKORUNUZ (Ayrı Özelliklerle): 0.63215
-------------------------------------------

Test verisi hazırlanıyor ve Kaggle için submission dosyası oluşturuluyor...

Oluşturulan submission satır sayısı: 18589 (Beklenen: 18589)
✅ Satır sayısı doğru.

Submission dosyası 'submission.csv' olarak kaydedildi.


**Data Sızıntısını Engelleyerek Yeni Eğitim**

In [ ]:
import polars as pl
import pandas as pd
import lightgbm as lgb
import numpy as np
import os

print("--- KOD 1: Model Eğitme (Fiyat Özellikleri ÇIKARILDI, Zamansal Özellikler KALDI) ---")

# --- Adım 1: Veri Yükleme ---
TRAIN_PATH = "/content/Trendyol Veri/enriched_train_data_FINAL.parquet"
print(f"Eğitim verisi yükleniyor: {TRAIN_PATH}")
train_full_df = pl.read_parquet(TRAIN_PATH)
print(f"Toplam eğitim verisi boyutu: {train_full_df.shape}")


# --- Adım 2: Ön Özellikler (Zamansal) ve Train/Validation Ayrımı ---
print("Zamansal özellikler üretiliyor ve veri zamana göre ayrılıyor...")
if "ts_hour" in train_full_df.columns and train_full_df["ts_hour"].dtype != pl.Datetime:
    train_full_df = train_full_df.with_columns(pl.col("ts_hour").str.to_datetime())

train_full_df = train_full_df.with_columns(
    pl.col("ts_hour").dt.hour().alias("hour_of_day"),
    pl.col("ts_hour").dt.weekday().alias("day_of_week")
)

train_full_df = train_full_df.sort("ts_hour")
split_index = int(len(train_full_df) * 0.85)
train_df, val_df = train_full_df[:split_index], train_full_df[split_index:]
print(f"Train parçası boyutu: {train_df.shape}")
print(f"Validation parçası boyutu: {val_df.shape}")


# --- Adım 3: Fiyat Etkileşim Özellikleri ÇIKARILDI ---
# Bu adımda artık kullanıcı ortalama fiyatı hesaplanmıyor ve birleştirilmiyor.
print("✅ Fiyat etkileşim özellikleri bu denemede kullanılmayacaktır.")


# --- Adım 4: Veri Hazırlama ve Tip Dönüşümü ---
print("Veri setleri Pandas'a çevriliyor ve model için hazırlanıyor...")

train_pd = train_df.to_pandas()
val_pd = val_df.to_pandas()

# Sayısal sütunlardaki NaN/Inf değerlerini yönetme (artık fiyat oranları yok, genel fillna(0) yapabiliriz)
numeric_cols = train_pd.select_dtypes(include=np.number).columns.tolist()
for col in numeric_cols:
    # Sonsuz değerleri NaN yapıp sonra NaN'ları 0 ile dolduralım
    train_pd[col] = train_pd[col].replace([np.inf, -np.inf], np.nan).fillna(0)
    val_pd[col] = val_pd[col].replace([np.inf, -np.inf], np.nan).fillna(0)


string_cols = [col for col in train_pd.columns if train_pd[col].dtype == 'object']
temporal_cols_as_cat = ["hour_of_day", "day_of_week"]
cols_to_categorize = string_cols + temporal_cols_as_cat
for col in cols_to_categorize:
    if col in train_pd.columns:
        train_pd[col] = train_pd[col].astype('category')
        val_pd[col] = val_pd[col].astype('category')


targets = ["ordered", "clicked"]
# Düzeltme: exclude_cols listesinden price_vs_user_avg_ratio ve user_avg_order_price çıkarıldı
exclude_cols = targets + ["added_to_cart", "added_to_fav", "ts_hour", "session_id", "user_id_hashed", "content_id_hashed", "search_term_normalized", "level2_category_name", "level1_category_name", "cv_tags", "update_date", "content_creation_date", "price_vs_user_avg_ratio", "user_avg_order_price"]
features = [col for col in train_pd.columns if col not in exclude_cols]
cat_features = [col for col in features if train_pd[col].dtype.name == 'category']

print(f"Kullanılacak özellik sayısı: {len(features)}")
print(f"Güncel Kategorik özellik sayısı: {len(cat_features)}")


# --- Adım 5: lgb.Dataset Oluşturma ---
print("lgb.Dataset nesneleri oluşturuluyor...")
# Categorical features listesini yeniden kontrol edelim
cat_features = [col for col in features if train_pd[col].dtype.name == 'category']
print(f"Son Kategorik özellik sayısı: {len(cat_features)}")

lgb_train_ordered = lgb.Dataset(train_pd[features], label=train_pd['ordered'], categorical_feature=cat_features if cat_features else None)
lgb_valid_ordered = lgb.Dataset(val_pd[features], label=val_pd['ordered'], categorical_feature=cat_features if cat_features else None)

lgb_train_clicked = lgb.Dataset(train_pd[features], label=train_pd['clicked'], categorical_feature=cat_features if cat_features else None)
lgb_valid_clicked = lgb.Dataset(val_pd[features], label=val_pd['clicked'], categorical_feature=cat_features if cat_features else None)


# --- Adım 6: Model Eğitimi ve Kaydetme ---
params = { "objective": "binary", "metric": "auc", "boosting_type": "gbdt", "learning_rate": 0.03, "num_leaves": 64, "verbose": -1, "seed": 42 }
callbacks = [lgb.early_stopping(100, verbose=True)]
MODEL_DIR = "models_4_doldur_FiyatUygunluk" # Klasör adını değiştirmedim, üzerine yazacak
os.makedirs(MODEL_DIR, exist_ok=True)

print("\n'ordered' modeli eğitiliyor...")
ordered_counts = train_pd['ordered'].value_counts()
params['scale_pos_weight'] = ordered_counts[0] / ordered_counts[1]
model_ordered = lgb.train(params, lgb_train_ordered, valid_sets=[lgb_train_ordered, lgb_valid_ordered], valid_names=["train", "valid"], num_boost_round=2000, callbacks=callbacks)
model_ordered_path = os.path.join(MODEL_DIR, "model_ordered_v6_no_price_features.txt") # Dosya adını güncelledim
model_ordered.save_model(model_ordered_path)
print(f"✅ 'ordered' modeli '{model_ordered_path}' olarak kaydedildi.")

print("\n'clicked' modeli eğitiliyor...")
clicked_counts = train_pd['clicked'].value_counts()
params['scale_pos_weight'] = clicked_counts[0] / clicked_counts[1]
model_clicked = lgb.train(params, lgb_train_clicked, valid_sets=[lgb_train_clicked, lgb_valid_clicked], valid_names=["train", "valid"], num_boost_round=2000, callbacks=callbacks)
model_clicked_path = os.path.join(MODEL_DIR, "model_clicked_v6_no_price_features.txt") # Dosya adını güncelledim
model_clicked.save_model(model_clicked_path)
print(f"✅ 'clicked' modeli '{model_clicked_path}' olarak kaydedildi.")

print("\n--- Eğitim ve Kaydetme İşlemi Tamamlandı! ---")

--- KOD 1: Model Eğitme (Fiyat Özellikleri ÇIKARILDI, Zamansal Özellikler KALDI) ---
Eğitim verisi yükleniyor: /content/Trendyol Veri/enriched_train_data_FINAL.parquet
Toplam eğitim verisi boyutu: (2773805, 37)
Zamansal özellikler üretiliyor ve veri zamana göre ayrılıyor...
Train parçası boyutu: (2357734, 39)
Validation parçası boyutu: (416071, 39)
✅ Fiyat etkileşim özellikleri bu denemede kullanılmayacaktır.
Veri setleri Pandas'a çevriliyor ve model için hazırlanıyor...
Kullanılacak özellik sayısı: 25
Güncel Kategorik özellik sayısı: 4
lgb.Dataset nesneleri oluşturuluyor...
Son Kategorik özellik sayısı: 4

'ordered' modeli eğitiliyor...
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[1]	train's auc: 0.764107	valid's auc: 0.700363
✅ 'ordered' modeli 'models_4_doldur_FiyatUygunluk/model_ordered_v6_no_price_features.txt' olarak kaydedildi.

'clicked' modeli eğitiliyor...
Training until validation scores don't improve for 100 rounds
Early 

**Kod 2** data sızıntısı giderilmiş

In [ ]:
import polars as pl
import pandas as pd
import lightgbm as lgb
import numpy as np
import os
from trendyol_metric_group_auc import score

print("--- KOD 2: Nihai Tahmin ve Skorlama (Fiyat Özellikleri ÇIKARILMIŞ) ---")

# --- Adım 1: Gerekli Dosyaları Yükleme ---
VAL_PATH = "/content/Trendyol Veri/enriched_train_data_FINAL.parquet"
TEST_PATH = "/content/Trendyol Veri/enriched_test_data_FINAL.parquet"
MODELS_DIR = "models_4_doldur_FiyatUygunluk" # Model klasörü aynı kaldı
# Düzeltme: Model dosya adlarını güncelledik (no_price_features versiyonları)
MODEL_ORDERED_PATH = os.path.join(MODELS_DIR, 'model_ordered_v6_no_price_features.txt')
MODEL_CLICKED_PATH = os.path.join(MODELS_DIR, 'model_clicked_v6_no_price_features.txt')


print("Veriler ve kaydedilmiş v6 modelleri yükleniyor...")
val_full_df = pl.read_parquet(VAL_PATH)
test_df_pl = pl.read_parquet(TEST_PATH)
model_ordered = lgb.Booster(model_file=MODEL_ORDERED_PATH)
model_clicked = lgb.Booster(model_file=MODEL_CLICKED_PATH)
print("✅ Yükleme tamamlandı.")


# --- Adım 2: Lokal Skor İçin Validation Seti Hazırlama ---
print("\nLokal skor için validation seti hazırlanıyor...")
if "ts_hour" in val_full_df.columns and val_full_df["ts_hour"].dtype != pl.Datetime:
    val_full_df = val_full_df.with_columns(pl.col("ts_hour").str.to_datetime())

# Zamansal özellikleri ekle (Bunlar hala kullanılıyor)
val_full_df = val_full_df.with_columns(
    pl.col("ts_hour").dt.hour().alias("hour_of_day"),
    pl.col("ts_hour").dt.weekday().alias("day_of_week")
)

# Düzeltme: Fiyat özelliği ekleme ve işleme adımlarını ÇIKARDIK


val_full_df = val_full_df.sort("ts_hour")
split_index = int(len(val_full_df) * 0.85)
# Pandas'a çevir ve sayısal null'ları 0 doldur (fiyat özellikleri artık olmadığı için genel fillna(0) güvenli)
val_pd = val_full_df[split_index:].to_pandas()

# Sayısal sütunlardaki NaN/Inf değerlerini yönetme ve 0 ile doldurma
numeric_cols = val_pd.select_dtypes(include=np.number).columns.tolist()
for col in numeric_cols:
    val_pd[col] = val_pd[col].replace([np.inf, -np.inf], np.nan).fillna(0)


targets = ["ordered", "clicked"]
# Düzeltme: exclude_cols listesi eğitimdekiyle aynı (fiyat özellikleri çıkarılmış hali)
exclude_cols = targets + ["added_to_cart", "added_to_fav", "ts_hour", "session_id", "user_id_hashed", "content_id_hashed", "search_term_normalized", "level2_category_name", "level1_category_name", "cv_tags", "update_date", "content_creation_date", "price_vs_user_avg_ratio", "user_avg_order_price"]
features = [col for col in val_pd.columns if col not in exclude_cols] # Düzeltme: features listesi exclude_cols'a göre güncellendi

string_cols = [col for col in val_pd.columns if val_pd[col].dtype == 'object']
temporal_cols_as_cat = ["hour_of_day", "day_of_week"]
cols_to_categorize = string_cols + temporal_cols_as_cat
for col in cols_to_categorize:
    if col in val_pd.columns:
        val_pd[col] = val_pd[col].astype('category')

# Tahmin üret
val_pd['p_order'] = model_ordered.predict(val_pd[features], num_iteration=model_ordered.best_iteration)
val_pd['p_click'] = model_clicked.predict(val_pd[features], num_iteration=model_clicked.best_iteration)
val_pd['final_score'] = 0.7 * val_pd['p_order'] + 0.3 * val_pd['p_click']

# Skor fonksiyonu için gerekli DataFrame'leri oluştur
val_solution = val_pd.groupby('session_id', observed=False).agg(ordered_items=('ordered', lambda x: ' '.join(val_pd.loc[x.index][x == 1]['content_id_hashed'])), clicked_items=('clicked', lambda x: ' '.join(val_pd.loc[x.index][x == 1]['content_id_hashed'])), all_items=('content_id_hashed', ' '.join)).reset_index()
val_submission = val_pd.sort_values(['session_id', 'final_score'], ascending=[True, False]).groupby('session_id', observed=False)['content_id_hashed'].apply(' '.join).reset_index()
val_submission.rename(columns={'content_id_hashed': 'prediction'}, inplace=True)

try:
    local_final_score = score(val_solution, val_submission, 'session_id')
    print("\n-------------------------------------------")
    print(f"🏆 LOKAL SKORUNUZ (Fiyat Özellikleri Çıkarılmış): {local_final_score:.5f}")
    print("-------------------------------------------")
except Exception as e:
    print(f"Skor hesaplanırken bir hata oluştu: {e}")


# --- Adım 3: Kaggle için Submission Dosyası Oluşturma ---
print("\nTest verisi hazırlanıyor ve Kaggle için submission dosyası oluşturuluyor...")
if "ts_hour" in test_df_pl.columns and test_df_pl["ts_hour"].dtype != pl.Datetime:
    test_df_pl = test_df_pl.with_columns(pl.col("ts_hour").str.to_datetime())
# Zamansal özellikleri ekle (Bunlar hala kullanılıyor)
test_df_pl = test_df_pl.with_columns(
    pl.col("ts_hour").dt.hour().alias("hour_of_day"),
    pl.col("ts_hour").dt.weekday().alias("day_of_week")
)
# Düzeltme: Fiyat özelliği ekleme ve işleme adımlarını ÇIKARDIK

# Pandas'a çevir ve sayısal null'ları 0 doldur
test_pd = test_df_pl.to_pandas()
numeric_cols_test = test_pd.select_dtypes(include=np.number).columns.tolist()
for col in numeric_cols_test:
     test_pd[col] = test_pd[col].replace([np.inf, -np.inf], np.nan).fillna(0)


for col in cols_to_categorize:
    if col in test_pd.columns:
        test_pd[col] = test_pd[col].astype('category')

# Features listesi eğitimdekiyle aynı olmalı
# exclude_cols listesi eğitimdekiyle aynı (fiyat özellikleri çıkarılmış hali)
exclude_cols_test = targets + ["added_to_cart", "added_to_fav", "ts_hour", "session_id", "user_id_hashed", "content_id_hashed", "search_term_normalized", "level2_category_name", "level1_category_name", "cv_tags", "update_date", "content_creation_date", "price_vs_user_avg_ratio", "user_avg_order_price"]
features_test = [col for col in test_pd.columns if col not in exclude_cols_test] # Düzeltme: features_test listesi exclude_cols_test'e göre güncellendi

p_order_test = model_ordered.predict(test_pd[features_test], num_iteration=model_ordered.best_iteration)
p_click_test = model_clicked.predict(test_pd[features_test], num_iteration=model_clicked.best_iteration)


test_df_pl = test_df_pl.with_columns(
    final_score=(0.7 * pl.Series(p_order_test)) + (0.3 * pl.Series(p_click_test))
)
submission_df = test_df_pl.sort(["session_id", "final_score"], descending=True).group_by("session_id").agg(
    pl.col("content_id_hashed").alias("prediction")
).with_columns(
    pl.col("prediction").list.join(" ")
)

expected_rows = 18589
actual_rows = submission_df.shape[0]
print(f"\nOluşturulan submission satır sayısı: {actual_rows} (Beklenen: {expected_rows})")
if actual_rows == expected_rows: print("✅ Satır sayısı doğru.")
else: print("❌ UYARI: Satır sayısı yanlış!")

submission_path = "submission_no_price_features.csv" # Dosya adını güncelledim
submission_df.write_csv(submission_path)
print(f"\nSubmission dosyası '{submission_path}' olarak kaydedildi.")

--- KOD 2: Nihai Tahmin ve Skorlama (Fiyat Özellikleri ÇIKARILMIŞ) ---
Veriler ve kaydedilmiş v6 modelleri yükleniyor...
✅ Yükleme tamamlandı.

Lokal skor için validation seti hazırlanıyor...
Ordered AUC:  0.6504243273845817
Clicked AUC:  0.5914654200400459

-------------------------------------------
🏆 LOKAL SKORUNUZ (Fiyat Özellikleri Çıkarılmış): 0.63274
-------------------------------------------

Test verisi hazırlanıyor ve Kaggle için submission dosyası oluşturuluyor...

Oluşturulan submission satır sayısı: 18589 (Beklenen: 18589)
✅ Satır sayısı doğru.

Submission dosyası 'submission_no_price_features.csv' olarak kaydedildi.


# Deneme 4 Alaka Düzeyi

**Kod 1** Alaka Düzeyine göre eğitmek ve modelleri kaydetmek

In [ ]:
import polars as pl
import pandas as pd
import lightgbm as lgb
import numpy as np
import os

print("--- KOD 1: Model Eğitme (Yeni Alaka Düzeyi Özellikleriyle) ---")

# --- Adım 1: Veri Yükleme ---
TRAIN_PATH = "/content/Trendyol Veri/enriched_train_data_FINAL.parquet"
train_full_df = pl.read_parquet(TRAIN_PATH)
print(f"Toplam eğitim verisi boyutu: {train_full_df.shape}")

# --- Adım 2: Özellik Mühendisliği ve Veri Ayırma ---
print("Ön özellikler üretiliyor...")
if "ts_hour" in train_full_df.columns and train_full_df["ts_hour"].dtype != pl.Datetime:
    train_full_df = train_full_df.with_columns(pl.col("ts_hour").str.to_datetime())

# 2.1 Zamansal Özellikler
train_full_df = train_full_df.with_columns(
    pl.col("ts_hour").dt.hour().alias("hour_of_day"),
    pl.col("ts_hour").dt.weekday().alias("day_of_week")
)

# 2.2 Alaka Düzeyi (Relevance) Özellikleri
print("Yeni Alaka Düzeyi (Relevance) özellikleri üretiliyor...")
train_full_df = train_full_df.with_columns(pl.col("search_term_normalized").fill_null(""), pl.col("cv_tags").fill_null(""))
search_words = pl.col("search_term_normalized").str.split(" ")
tag_words = pl.col("cv_tags").str.split(" ")
intersection_len = search_words.list.set_intersection(tag_words).list.len()
union_len = search_words.list.set_union(tag_words).list.len()
train_full_df = train_full_df.with_columns(
    term_in_tags_match = (intersection_len > 0).cast(pl.Int8),
    jaccard_similarity_tags = pl.when(union_len > 0).then(intersection_len / union_len).otherwise(0.0)
)

# 2.3 Train/Validation Ayrımı (ÖZELLİK ÜRETİMİNDEN ÖNCE)
train_full_df = train_full_df.sort("ts_hour")
split_index = int(len(train_full_df) * 0.85)
train_df, val_df = train_full_df[:split_index], train_full_df[split_index:]
print(f"Train parçası boyutu: {train_df.shape}")
print(f"Validation parçası boyutu: {val_df.shape}")

# 2.4 Fiyat Etkileşim Özelliğini SADECE Train Setinden Hesaplama
print("Fiyat etkileşim özelliği, SADECE train verisi üzerinden hesaplanıyor...")
price_features_lazy = pl.scan_parquet(TRAIN_PATH).select(["content_id_hashed", "selling_price", "update_date"])
latest_price_features = price_features_lazy.sort("update_date", descending=True).group_by("content_id_hashed").first().collect()
user_orders = train_df.filter(pl.col("ordered") == 1).select(["user_id_hashed", "content_id_hashed"])
user_orders_with_price = user_orders.join(latest_price_features, on="content_id_hashed", how="left")
agg_user_price_features = user_orders_with_price.group_by("user_id_hashed").agg(pl.mean("selling_price").alias("user_avg_order_price"))
FEATURES_DIR = "features"
os.makedirs(FEATURES_DIR, exist_ok=True)
agg_user_price_features.write_parquet(os.path.join(FEATURES_DIR, "user_price_features_final.parquet"))
print(f"✅ Sızıntısız kullanıcı fiyat özellikleri kaydedildi.")

# 2.5 Özellikleri Ekleme ve Akıllı Boşluk Doldurma
global_avg_order_price = agg_user_price_features.select(pl.mean("user_avg_order_price")).item()
train_df = train_df.join(agg_user_price_features, on="user_id_hashed", how="left").with_columns(pl.col("user_avg_order_price").fill_null(global_avg_order_price)).with_columns((pl.col("selling_price") / pl.col("user_avg_order_price")).alias("price_vs_user_avg_ratio"))
val_df = val_df.join(agg_user_price_features, on="user_id_hashed", how="left").with_columns(pl.col("user_avg_order_price").fill_null(global_avg_order_price)).with_columns((pl.col("selling_price") / pl.col("user_avg_order_price")).alias("price_vs_user_avg_ratio"))

# --- Adım 3: Veri Hazırlama ve AYRI ÖZELLİK SETLERİ OLUŞTURMA ---
print("Veri setleri Pandas'a çevriliyor ve ayrı özellik setleri oluşturuluyor...")
train_pd = train_df.to_pandas().fillna(0)
val_pd = val_df.to_pandas().fillna(0)
string_cols = [col for col in train_pd.columns if train_pd[col].dtype == 'object']
temporal_cols_as_cat = ["hour_of_day", "day_of_week"]
cols_to_categorize = string_cols + temporal_cols_as_cat
for col in cols_to_categorize:
    if col in train_pd.columns:
        train_pd[col], val_pd[col] = train_pd[col].astype('category'), val_pd[col].astype('category')

targets = ["ordered", "clicked"]
exclude_cols = targets + ["added_to_cart", "added_to_fav", "ts_hour", "session_id", "user_id_hashed", "content_id_hashed", "search_term_normalized", "level2_category_name", "level1_category_name", "cv_tags", "update_date", "content_creation_date"]
features_base = [col for col in train_pd.columns if col not in exclude_cols]
price_features_to_exclude = ["user_avg_order_price", "price_vs_user_avg_ratio"]
features_for_ordered = [f for f in features_base if f not in price_features_to_exclude]
features_for_clicked = features_base
cat_features_ordered = [f for f in features_for_ordered if isinstance(train_pd[f].dtype, pd.CategoricalDtype)]
cat_features_clicked = [f for f in features_for_clicked if isinstance(train_pd[f].dtype, pd.CategoricalDtype)]
print(f"'ordered' modeli için özellik sayısı: {len(features_for_ordered)}")
print(f"'clicked' modeli için özellik sayısı: {len(features_for_clicked)}")

# --- Adım 4: lgb.Dataset Oluşturma ---
print("Modele özel lgb.Dataset nesneleri oluşturuluyor...")
lgb_train_ordered = lgb.Dataset(train_pd[features_for_ordered], label=train_pd['ordered'], categorical_feature=cat_features_ordered)
lgb_valid_ordered = lgb.Dataset(val_pd[features_for_ordered], label=val_pd['ordered'], categorical_feature=cat_features_ordered)
lgb_train_clicked = lgb.Dataset(train_pd[features_for_clicked], label=train_pd['clicked'], categorical_feature=cat_features_clicked)
lgb_valid_clicked = lgb.Dataset(val_pd[features_for_clicked], label=val_pd['clicked'], categorical_feature=cat_features_clicked)

# --- Adım 5: Model Eğitimi ve Kaydetme ---
params = { "objective": "binary", "metric": "auc", "boosting_type": "gbdt", "learning_rate": 0.03, "num_leaves": 64, "verbose": -1, "seed": 42 }
callbacks = [lgb.early_stopping(100, verbose=True)]
MODEL_DIR = "models_6_doldur_AlakaDuzeyi"
os.makedirs(MODEL_DIR, exist_ok=True)

print("\n'ordered' modeli (sınırlı özelliklerle) eğitiliyor...")
ordered_counts = train_pd['ordered'].value_counts()
params['scale_pos_weight'] = ordered_counts[0] / ordered_counts[1]
model_ordered = lgb.train(params, lgb_train_ordered, valid_sets=[lgb_train_ordered, lgb_valid_ordered], valid_names=["train", "valid"], num_boost_round=2000, callbacks=callbacks)
model_ordered.save_model(os.path.join(MODEL_DIR, "model_ordered_v6_relevance.txt"))
print(f"✅ 'ordered' modeli '{MODEL_DIR}/model_ordered_v6_relevance.txt' olarak kaydedildi.")

print("\n'clicked' modeli (tüm özelliklerle) eğitiliyor...")
clicked_counts = train_pd['clicked'].value_counts()
params['scale_pos_weight'] = clicked_counts[0] / clicked_counts[1]
model_clicked = lgb.train(params, lgb_train_clicked, valid_sets=[lgb_train_clicked, lgb_valid_clicked], valid_names=["train", "valid"], num_boost_round=2000, callbacks=callbacks)
model_clicked.save_model(os.path.join(MODEL_DIR, "model_clicked_v6_relevance.txt"))
print(f"✅ 'clicked' modeli '{MODEL_DIR}/model_clicked_v6_relevance.txt' olarak kaydedildi.")

print("\n--- Eğitim ve Kaydetme İşlemi Tamamlandı! ---")

--- KOD 1: Model Eğitme (Yeni Alaka Düzeyi Özellikleriyle) ---
Toplam eğitim verisi boyutu: (2773805, 37)
Ön özellikler üretiliyor...
Yeni Alaka Düzeyi (Relevance) özellikleri üretiliyor...
Train parçası boyutu: (2357734, 41)
Validation parçası boyutu: (416071, 41)
Fiyat etkileşim özelliği, SADECE train verisi üzerinden hesaplanıyor...
✅ Sızıntısız kullanıcı fiyat özellikleri kaydedildi.
Veri setleri Pandas'a çevriliyor ve ayrı özellik setleri oluşturuluyor...
'ordered' modeli için özellik sayısı: 27
'clicked' modeli için özellik sayısı: 29
Modele özel lgb.Dataset nesneleri oluşturuluyor...

'ordered' modeli (sınırlı özelliklerle) eğitiliyor...
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[1]	train's auc: 0.764107	valid's auc: 0.700363
✅ 'ordered' modeli 'models_6_doldur_AlakaDuzeyi/model_ordered_v6_relevance.txt' olarak kaydedildi.

'clicked' modeli (tüm özelliklerle) eğitiliyor...
Training until validation scores don't improve for 1

**Kod 2** Alaka Düzeyi ile test etme ve submissions oluşturma

In [ ]:
import polars as pl
import pandas as pd
import lightgbm as lgb
import numpy as np
import os
from trendyol_metric_group_auc import score

print("--- KOD 2: Nihai Tahmin ve Skorlama (Yeni Alaka Düzeyi Özellikleriyle) ---")

# --- Adım 1: Gerekli Dosyaları Yükleme ---
VAL_PATH = "/content/Trendyol Veri/enriched_train_data_FINAL.parquet"
TEST_PATH = "/content/Trendyol Veri/enriched_test_data_FINAL.parquet"
FEATURES_DIR = "features"
MODELS_DIR = "models_6_doldur_AlakaDuzeyi"
USER_PRICE_FEATURES_PATH = os.path.join(FEATURES_DIR, "user_price_features_final.parquet")
MODEL_ORDERED_PATH = os.path.join(MODELS_DIR, 'model_ordered_v6_relevance.txt')
MODEL_CLICKED_PATH = os.path.join(MODELS_DIR, 'model_clicked_v6_relevance.txt')

print("Veriler, özellikler ve kaydedilmiş nihai modeller yükleniyor...")
val_full_df = pl.read_parquet(VAL_PATH)
test_df_pl = pl.read_parquet(TEST_PATH)
user_price_features = pl.read_parquet(USER_PRICE_FEATURES_PATH)
model_ordered = lgb.Booster(model_file=MODEL_ORDERED_PATH)
model_clicked = lgb.Booster(model_file=MODEL_CLICKED_PATH)
print("✅ Yükleme tamamlandı.")

# --- Adım 2: Lokal Skor İçin Validation Seti Hazırlama ---
print("\nLokal skor için validation seti hazırlanıyor...")
# 2.1 Tüm Özellikleri Ekle
if "ts_hour" in val_full_df.columns and val_full_df["ts_hour"].dtype != pl.Datetime:
    val_full_df = val_full_df.with_columns(pl.col("ts_hour").str.to_datetime())
val_full_df = val_full_df.with_columns(pl.col("ts_hour").dt.hour().alias("hour_of_day"), pl.col("ts_hour").dt.weekday().alias("day_of_week"))
val_full_df = val_full_df.with_columns(pl.col("search_term_normalized").fill_null(""), pl.col("cv_tags").fill_null(""))
search_words_val = pl.col("search_term_normalized").str.split(" ")
tag_words_val = pl.col("cv_tags").str.split(" ")
intersection_len_val = search_words_val.list.set_intersection(tag_words_val).list.len()
union_len_val = search_words_val.list.set_union(tag_words_val).list.len()
val_full_df = val_full_df.with_columns(term_in_tags_match=(intersection_len_val > 0).cast(pl.Int8), jaccard_similarity_tags=pl.when(union_len_val > 0).then(intersection_len_val / union_len_val).otherwise(0.0))
global_avg_order_price = user_price_features.select(pl.mean("user_avg_order_price")).item()
val_full_df = val_full_df.join(user_price_features, on="user_id_hashed", how="left").with_columns(pl.col("user_avg_order_price").fill_null(global_avg_order_price)).with_columns((pl.col("selling_price") / pl.col("user_avg_order_price")).alias("price_vs_user_avg_ratio"))

# 2.2 Veriyi Ayır ve Hazırla
val_full_df = val_full_df.sort("ts_hour")
split_index = int(len(val_full_df) * 0.85)
val_pd = val_full_df[split_index:].to_pandas().fillna(0)

# 2.3 Ayrı Özellik Setlerini Tanımla
targets = ["ordered", "clicked"]
exclude_cols = targets + ["added_to_cart", "added_to_fav", "ts_hour", "session_id", "user_id_hashed", "content_id_hashed", "search_term_normalized", "level2_category_name", "level1_category_name", "cv_tags", "update_date", "content_creation_date"]
features_base = [col for col in val_pd.columns if col not in exclude_cols]
price_features_to_exclude = ["user_avg_order_price", "price_vs_user_avg_ratio"]
features_for_ordered = [f for f in features_base if f not in price_features_to_exclude]
features_for_clicked = features_base
string_cols = [col for col in val_pd.columns if val_pd[col].dtype == 'object']
temporal_cols_as_cat = ["hour_of_day", "day_of_week"]
cols_to_categorize = string_cols + temporal_cols_as_cat
for col in cols_to_categorize:
    if col in val_pd.columns:
        val_pd[col] = val_pd[col].astype('category')

# 2.4 Tahmin ve Skorlama
val_pd['p_order'] = model_ordered.predict(val_pd[features_for_ordered], num_iteration=model_ordered.best_iteration)
val_pd['p_click'] = model_clicked.predict(val_pd[features_for_clicked], num_iteration=model_clicked.best_iteration)
val_pd['final_score'] = 0.7 * val_pd['p_order'] + 0.3 * val_pd['p_click']
val_solution = val_pd.groupby('session_id').agg(ordered_items=('ordered', lambda x: ' '.join(val_pd.loc[x.index][x == 1]['content_id_hashed'])), clicked_items=('clicked', lambda x: ' '.join(val_pd.loc[x.index][x == 1]['content_id_hashed'])), all_items=('content_id_hashed', ' '.join)).reset_index()
val_submission = val_pd.sort_values(['session_id', 'final_score'], ascending=[True, False]).groupby('session_id')['content_id_hashed'].apply(' '.join).reset_index()
val_submission.rename(columns={'content_id_hashed': 'prediction'}, inplace=True)
try:
    local_final_score = score(val_solution, val_submission, 'session_id')
    print("\n-------------------------------------------")
    print(f"🏆 LOKAL SKORUNUZ (Yeni Özelliklerle): {local_final_score:.5f}")
    print("-------------------------------------------")
except Exception as e:
    print(f"Skor hesaplanırken bir hata oluştu: {e}")

# --- Adım 3: Kaggle için Submission Dosyası Oluşturma ---
print("\nTest verisi hazırlanıyor ve Kaggle için submission dosyası oluşturuluyor...")
if "ts_hour" in test_df_pl.columns and test_df_pl["ts_hour"].dtype != pl.Datetime:
    test_df_pl = test_df_pl.with_columns(pl.col("ts_hour").str.to_datetime())
test_df_pl = test_df_pl.with_columns(pl.col("ts_hour").dt.hour().alias("hour_of_day"), pl.col("ts_hour").dt.weekday().alias("day_of_week"))
test_df_pl = test_df_pl.with_columns(pl.col("search_term_normalized").fill_null(""), pl.col("cv_tags").fill_null(""))
search_words_test = pl.col("search_term_normalized").str.split(" ")
tag_words_test = pl.col("cv_tags").str.split(" ")
intersection_len_test = search_words_test.list.set_intersection(tag_words_test).list.len()
union_len_test = search_words_test.list.set_union(tag_words_test).list.len()
test_df_pl = test_df_pl.with_columns(term_in_tags_match=(intersection_len_test > 0).cast(pl.Int8), jaccard_similarity_tags=pl.when(union_len_test > 0).then(intersection_len_test / union_len_test).otherwise(0.0))
test_df_pl = test_df_pl.join(user_price_features, on="user_id_hashed", how="left").with_columns(pl.col("user_avg_order_price").fill_null(global_avg_order_price)).with_columns((pl.col("selling_price") / pl.col("user_avg_order_price")).alias("price_vs_user_avg_ratio"))

test_pd = test_df_pl.to_pandas().fillna(0)
for col in cols_to_categorize:
    if col in test_pd.columns:
        test_pd[col] = test_pd[col].astype('category')

p_order_test = model_ordered.predict(test_pd[features_for_ordered], num_iteration=model_ordered.best_iteration)
p_click_test = model_clicked.predict(test_pd[features_for_clicked], num_iteration=model_clicked.best_iteration)

test_df_pl = test_df_pl.with_columns(final_score=(0.7 * pl.Series(p_order_test)) + (0.3 * pl.Series(p_click_test)))
submission_df = test_df_pl.sort(["session_id", "final_score"], descending=True).group_by("session_id").agg(pl.col("content_id_hashed").alias("prediction")).with_columns(pl.col("prediction").list.join(" "))
expected_rows = 18589
actual_rows = submission_df.shape[0]
print(f"\nOluşturulan submission satır sayısı: {actual_rows} (Beklenen: {expected_rows})")
if actual_rows == expected_rows: print("✅ Satır sayısı doğru.")
else: print("❌ UYARI: Satır sayısı yanlış!")
submission_path = "submission.csv"
submission_df.write_csv(submission_path)
print(f"\nSubmission dosyası '{submission_path}' olarak kaydedildi.")

--- KOD 2: Nihai Tahmin ve Skorlama (Yeni Alaka Düzeyi Özellikleriyle) ---
Veriler, özellikler ve kaydedilmiş nihai modeller yükleniyor...
✅ Yükleme tamamlandı.

Lokal skor için validation seti hazırlanıyor...


/tmp/ipython-input-1787726108.py:65: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  val_solution = val_pd.groupby('session_id').agg(ordered_items=('ordered', lambda x: ' '.join(val_pd.loc[x.index][x == 1]['content_id_hashed'])), clicked_items=('clicked', lambda x: ' '.join(val_pd.loc[x.index][x == 1]['content_id_hashed'])), all_items=('content_id_hashed', ' '.join)).reset_index()
/tmp/ipython-input-1787726108.py:66: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  val_submission = val_pd.sort_values(['session_id', 'final_score'], ascending=[True, False]).groupby('session_id')['content_id_hashed'].apply(' '.join).

Ordered AUC:  0.6501352808701953
Clicked AUC:  0.5901990785840607

-------------------------------------------
🏆 LOKAL SKORUNUZ (Yeni Özelliklerle): 0.63215
-------------------------------------------

Test verisi hazırlanıyor ve Kaggle için submission dosyası oluşturuluyor...

Oluşturulan submission satır sayısı: 18589 (Beklenen: 18589)
✅ Satır sayısı doğru.

Submission dosyası 'submission.csv' olarak kaydedildi.


# Deneme 5 Hiperparametre Optimizasyonu

**Kod 1** Optuna kullnarak hiperparemetre optimizasyonu yapmak

In [ ]:
!pip install optuna -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.9/395.9 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.0/247.0 kB 12.5 MB/s eta 0:00:00


In [ ]:
import polars as pl
import pandas as pd
import lightgbm as lgb
import numpy as np
import os
import optuna # Optuna kütüphanesini ekledik

print("--- KOD 1: Baseline Model İçin Optuna ile HPO ve Eğitim ---")

# --- Adım 1: Veri Yükleme ---
TRAIN_PATH = "/content/Trendyol Veri/enriched_train_data_FINAL.parquet"
print(f"Eğitim verisi yükleniyor: {TRAIN_PATH}")
train_full_df = pl.read_parquet(TRAIN_PATH)
print(f"Toplam eğitim verisi boyutu: {train_full_df.shape}")


# --- Adım 2: Veriyi Zamana Göre Ayırma (Train / Validation Split) ---
print("Eğitim verisi, zamana göre sıralanıp yüzdesel olarak ayrılıyor...")
if "ts_hour" in train_full_df.columns and train_full_df["ts_hour"].dtype != pl.Datetime:
    train_full_df = train_full_df.with_columns(pl.col("ts_hour").str.to_datetime())
train_full_df = train_full_df.sort("ts_hour")
split_index = int(len(train_full_df) * 0.85)
train_df = train_full_df[:split_index]
val_df = train_full_df[split_index:]
print(f"Train parçası boyutu: {train_df.shape}")
print(f"Validation parçası boyutu: {val_df.shape}")


# --- Adım 3: Veri Hazırlama ve Tip Dönüşümü ---
print("Veri setleri Pandas'a çevriliyor ve model için hazırlanıyor...")
train_pd = train_df.to_pandas().fillna(0)
val_pd = val_df.to_pandas().fillna(0)
for col in train_pd.columns:
    if train_pd[col].dtype == 'object':
        train_pd[col] = train_pd[col].astype('category')
        val_pd[col] = val_pd[col].astype('category')
targets = ["ordered", "clicked"]
exclude_cols = targets + ["added_to_cart", "added_to_fav", "ts_hour", "session_id", "user_id_hashed", "content_id_hashed", "search_term_normalized", "level2_category_name", "level1_category_name", "cv_tags", "update_date", "content_creation_date"]
features = [col for col in train_pd.columns if col not in exclude_cols]
cat_features = [col for col in features if isinstance(train_pd[col].dtype, pd.CategoricalDtype)]
print(f"Kullanılacak özellik sayısı: {len(features)}")


# --- YENİ Adım 4: Optuna Optimizasyon Fonksiyonu ('ordered' için) ---
# Bu fonksiyon, Optuna'nın her denemede çalıştıracağı model eğitim sürecini tanımlar
def objective_ordered(trial):
    # Optuna'nın deneyeceği hiperparametre aralıklarını belirliyoruz
    params = {
        "objective": "binary", "metric": "auc", "boosting_type": "gbdt",
        "seed": 42, "verbose": -1, "n_jobs": -1,
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1, log=True),
        'num_leaves': trial.suggest_int('num_leaves', 20, 100),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.6, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.6, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
        'lambda_l1': trial.suggest_float('lambda_l1', 1e-8, 10.0, log=True),
        'lambda_l2': trial.suggest_float('lambda_l2', 1e-8, 10.0, log=True),
    }
    # Sınıf dengesizliği için ağırlığı ekle
    ordered_counts = train_pd['ordered'].value_counts()
    params['scale_pos_weight'] = ordered_counts[0] / ordered_counts[1]

    # Modeli eğit
    lgb_train = lgb.Dataset(train_pd[features], label=train_pd['ordered'], categorical_feature=cat_features)
    lgb_valid = lgb.Dataset(val_pd[features], label=val_pd['ordered'], categorical_feature=cat_features)
    model = lgb.train(params, lgb_train, valid_sets=[lgb_valid], valid_names=["valid"], num_boost_round=2000, callbacks=[lgb.early_stopping(50, verbose=False)])

    # Optuna, bu döndürülen 'valid's auc' skorunu maksimize etmeye çalışacak
    return model.best_score['valid']['auc']

print("\n'ordered' modeli için hiperparametre optimizasyonu başlıyor...")
# Optimizasyon çalışmasını oluştur ve başlat
study_ordered = optuna.create_study(direction='maximize')
study_ordered.optimize(objective_ordered, n_trials=50) # Deneme sayısını ihtiyaca göre ayarlayabilirsiniz

# En iyi sonuçları al
best_params_ordered = study_ordered.best_params
print("En iyi 'ordered' parametreleri bulundu:", best_params_ordered)
print(f"En iyi validation AUC skoru: {study_ordered.best_value}")

# --- Adım 5: Nihai Modelleri En İyi Parametrelerle Eğitme ---
print("\nNihai modeller en iyi bulunan parametrelerle eğitiliyor...")
# Temel sabit parametreler
base_params = { "objective": "binary", "metric": "auc", "boosting_type": "gbdt", "verbose": -1, "seed": 42, "n_jobs": -1 }
callbacks = [lgb.early_stopping(100, verbose=True)]
MODEL_DIR = "models"
os.makedirs(MODEL_DIR, exist_ok=True)

# 5.1 'ordered' modeli (Optimize Edilmiş Parametrelerle)
final_params_ordered = base_params.copy()
final_params_ordered.update(best_params_ordered) # Bulunan en iyi parametreleri ekle
ordered_counts = train_pd['ordered'].value_counts()
final_params_ordered['scale_pos_weight'] = ordered_counts[0] / ordered_counts[1]
lgb_train_ord = lgb.Dataset(train_pd[features], label=train_pd['ordered'], categorical_feature=cat_features)
lgb_valid_ord = lgb.Dataset(val_pd[features], label=val_pd['ordered'], categorical_feature=cat_features)
model_ordered = lgb.train(final_params_ordered, lgb_train_ord, valid_sets=[lgb_train_ord, lgb_valid_ord], valid_names=["train", "valid"], num_boost_round=2000, callbacks=callbacks)
model_ordered.save_model(os.path.join(MODEL_DIR, "model_ordered_hpo_baseline.txt"))
print(f"✅ Optimize 'ordered' modeli '{MODEL_DIR}/model_ordered_hpo_baseline.txt' olarak kaydedildi.")

# 5.2 'clicked' modeli (Standart Parametrelerle)
print("\n'clicked' modeli eğitiliyor...")
final_params_clicked = base_params.copy()
final_params_clicked.update({'learning_rate': 0.03, 'num_leaves': 64}) # Standart iyi parametreler
clicked_counts = train_pd['clicked'].value_counts()
final_params_clicked['scale_pos_weight'] = clicked_counts[0] / clicked_counts[1]
lgb_train_clk = lgb.Dataset(train_pd[features], label=train_pd['clicked'], categorical_feature=cat_features)
lgb_valid_clk = lgb.Dataset(val_pd[features], label=val_pd['clicked'], categorical_feature=cat_features)
model_clicked = lgb.train(final_params_clicked, lgb_train_clk, valid_sets=[lgb_train_clk, lgb_valid_clk], valid_names=["train", "valid"], num_boost_round=2000, callbacks=callbacks)
model_clicked.save_model(os.path.join(MODEL_DIR, "model_clicked_standard_baseline.txt"))
print(f"✅ 'clicked' modeli '{MODEL_DIR}/model_clicked_standard_baseline.txt' olarak kaydedildi.")

print("\n--- Optimizasyon ve Eğitim İşlemi Tamamlandı! ---")

--- KOD 1: Baseline Model İçin Optuna ile HPO ve Eğitim ---
Eğitim verisi yükleniyor: /content/Trendyol Veri/enriched_train_data_FINAL.parquet
Toplam eğitim verisi boyutu: (2773805, 37)
Eğitim verisi, zamana göre sıralanıp yüzdesel olarak ayrılıyor...
Train parçası boyutu: (2357734, 37)
Validation parçası boyutu: (416071, 37)
Veri setleri Pandas'a çevriliyor ve model için hazırlanıyor...


[I 2025-08-10 15:17:21,958] A new study created in memory with name: no-name-c7e75471-104b-45e4-a704-d1eadef2bf47


Kullanılacak özellik sayısı: 23

'ordered' modeli için hiperparametre optimizasyonu başlıyor...


[I 2025-08-10 15:18:00,665] Trial 0 finished with value: 0.7013007336326083 and parameters: {'learning_rate': 0.014947202519089677, 'num_leaves': 73, 'feature_fraction': 0.7584936395519258, 'bagging_fraction': 0.81685896433614, 'bagging_freq': 6, 'lambda_l1': 0.16457017997277235, 'lambda_l2': 0.020673747840117242}. Best is trial 0 with value: 0.7013007336326083.
[I 2025-08-10 15:18:48,161] Trial 1 finished with value: 0.7119243462315256 and parameters: {'learning_rate': 0.025261338950755174, 'num_leaves': 40, 'feature_fraction': 0.7889317140090981, 'bagging_fraction': 0.9836379980755368, 'bagging_freq': 3, 'lambda_l1': 0.00026920731374345286, 'lambda_l2': 0.03238193256203181}. Best is trial 1 with value: 0.7119243462315256.
[I 2025-08-10 15:19:10,344] Trial 2 finished with value: 0.7043060733532196 and parameters: {'learning_rate': 0.060112489822934954, 'num_leaves': 26, 'feature_fraction': 0.6801168600829408, 'bagging_fraction': 0.6337996810831215, 'bagging_freq': 1, 'lambda_l1': 0.00

KeyboardInterrupt: 

**Kod 2** Test Etme

In [ ]:
import polars as pl
import pandas as pd
import lightgbm as lgb
import numpy as np
# trendyol_metric_group_auc.py dosyasının Colab ortamında olduğundan emin olun
from trendyol_metric_group_auc import score

print("--- KOD 2: Tahmin ve Skorlama Süreci Başladı ---")

# --- Adım 1: Gerekli Veri ve Kaydedilmiş Modelleri Yükleme ---
VAL_PATH = "/content/Trendyol Veri/enriched_train_data_FINAL.parquet"
TEST_PATH = "/content/Trendyol Veri/enriched_test_data_FINAL.parquet"
MODEL_ORDERED_PATH = '/content/models/model_ordered_hpo_baseline.txt'
MODEL_CLICKED_PATH = '/content/models/model_clicked_standard_baseline.txt'

print("Veriler ve kaydedilmiş modeller yükleniyor...")
val_full_df = pl.read_parquet(VAL_PATH)
test_df_pl = pl.read_parquet(TEST_PATH)
model_ordered = lgb.Booster(model_file=MODEL_ORDERED_PATH)
model_clicked = lgb.Booster(model_file=MODEL_CLICKED_PATH)
print("✅ Veriler ve modeller başarıyla yüklendi.")

# --- Adım 2: Validation Seti Hazırlama ve Lokal Skor Hesaplama ---
print("\nLokal skor için validation seti hazırlanıyor...")
if "ts_hour" in val_full_df.columns and val_full_df["ts_hour"].dtype != pl.Datetime:
    val_full_df = val_full_df.with_columns(pl.col("ts_hour").str.to_datetime())
val_full_df = val_full_df.sort("ts_hour")
split_index = int(len(val_full_df) * 0.85)
val_pd = val_full_df[split_index:].to_pandas().fillna(0)

# Özellik listesini eğitimdekiyle tutarlı olacak şekilde belirle
targets = ["ordered", "clicked"]
exclude_cols = targets + ["added_to_cart", "added_to_fav", "ts_hour", "session_id", "user_id_hashed", "content_id_hashed", "search_term_normalized", "level2_category_name", "level1_category_name", "cv_tags", "update_date", "content_creation_date"]
features = [col for col in val_pd.columns if col not in exclude_cols]
for col in features:
    if val_pd[col].dtype == 'object':
        val_pd[col] = val_pd[col].astype('category')

val_pd['p_order'] = model_ordered.predict(val_pd[features], num_iteration=model_ordered.best_iteration)
val_pd['p_click'] = model_clicked.predict(val_pd[features], num_iteration=model_clicked.best_iteration)
val_pd['final_score'] = 0.7 * val_pd['p_order'] + 0.3 * val_pd['p_click']

# Skor fonksiyonu için gerekli DataFrame'leri oluştur
val_solution = val_pd.groupby('session_id').agg(
    ordered_items=('ordered', lambda x: ' '.join(val_pd.loc[x.index][x == 1]['content_id_hashed'])),
    clicked_items=('clicked', lambda x: ' '.join(val_pd.loc[x.index][x == 1]['content_id_hashed'])),
    all_items=('content_id_hashed', ' '.join)
).reset_index()
val_submission = val_pd.sort_values(['session_id', 'final_score'], ascending=[True, False]).groupby('session_id')['content_id_hashed'].apply(' '.join).reset_index()
val_submission.rename(columns={'content_id_hashed': 'prediction'}, inplace=True)

try:
    local_final_score = score(val_solution, val_submission, 'session_id')
    print("\n-------------------------------------------")
    print(f"🏆 LOKAL SKORUNUZ: {local_final_score:.5f}")
    print("-------------------------------------------")
except Exception as e:
    print(f"Skor hesaplanırken bir hata oluştu: {e}")

# --- Adım 3: Kaggle için Submission Dosyası Oluşturma ---
print("\nTest verisi hazırlanıyor ve Kaggle için submission dosyası oluşturuluyor...")
test_pd = test_df_pl.to_pandas().fillna(0)
for col in features:
    if test_pd[col].dtype == 'object':
        test_pd[col] = test_pd[col].astype('category')

p_order_test = model_ordered.predict(test_pd[features], num_iteration=model_ordered.best_iteration)
p_click_test = model_clicked.predict(test_pd[features], num_iteration=model_clicked.best_iteration)

test_df_pl = test_df_pl.with_columns(
    final_score=(0.7 * pl.Series(p_order_test)) + (0.3 * pl.Series(p_click_test))
)

submission_df = test_df_pl.sort(["session_id", "final_score"], descending=True).group_by("session_id").agg(
    pl.col("content_id_hashed").alias("prediction")
).with_columns(
    pl.col("prediction").list.join(" ")
)

# Sağlama kontrolü
expected_rows = 18589
actual_rows = submission_df.shape[0]
print(f"\nOluşturulan submission satır sayısı: {actual_rows} (Beklenen: {expected_rows})")
if actual_rows == expected_rows: print("✅ Satır sayısı doğru.")
else: print("❌ UYARI: Satır sayısı yanlış!")

submission_path = "submission_final_v4.csv"
submission_df.write_csv(submission_path)
print(f"\nSubmission dosyası '{submission_path}' olarak kaydedildi.")